<a href="https://colab.research.google.com/github/drou0302/CapiPy/blob/main/CapiPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



>**WELCOME TO CAPIPY COLAB!**
---

In this colab file, you will be able to run the three most important modules of CapiPy and directly view your results. 
At the end, there is also the possibilty for you to download your project file to your local computer. 
All functionalities and files remain the same formatting (see https://github.com/drou0302/CapiPy for more information). 

The three modules that can be run are:

1.   MODELLER & 1.1 Quaternary model
2.   ACTIVE SITE IDENTIFICATION
3.   SURFACE ANALYSIS

To run each of the cells, just press the play button, wait and enjoy!

In the different modules, you will be asked to provide full paths to different files or folders. To do so, on the left hand side you have the access to your folder indicated by the 📁 symbol. By default, all the results are saved in the /content/ folder. To copy a path, select the file or folder and right click, then select 'Copy path' and paste directly that information. 

A new folder for each of your projects will be created when you run the MODELLER module. 

**Remember, that CapiPy is thought to be used sequentially, so follow the whole process for optimal results.**

In [ ]:
#@title DEPENDENCIES INSTALLATION
#@markdown Download all necessary dependencies for CapiPy and install them.
print('Downloading conda...')
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
print('Installing modeller...')
!conda install -c salilab modeller 2>&1 1>/dev/null

#@markdown Enter here your modeller license. You can obtain it from https://salilab.org/modeller/registration.html

MODELLER_LICENSE = "MODELIRANJE" #@param {type:"string"}

%cd /usr/local/lib/modeller-10.4/modlib/modeller/
with open('config.py', 'r+') as orig_config:
    with open('config1.py', 'w+') as new_config:
      for line in orig_config.readlines():
        if 'license' in line:
          new_config.write("license = r'" + MODELLER_LICENSE + "'")
        elif 'license' not in line:
          new_config.write(line)
%rm config.py
%mv config1.py config.py
%cd /content/

print('Installing biopython...')
!pip install biopython 2>&1 1>/dev/null
!pip install xmltramp2 2>&1 1>/dev/null
!pip install pandas 2>&1 1>/dev/null
!pip install py3Dmol 2>&1 1>/dev/null
!wget https://raw.githubusercontent.com/ebi-wp/webservice-clients/master/python/ncbiblast.py 2>&1 1>/dev/null


print('Installing clustalw2...') 
!wget http://www.clustal.org/download/current/clustalw-2.1.tar.gz 2>&1 1>/dev/null
!tar xvzf clustalw-2.1.tar.gz 2>&1 1>/dev/null
%cd clustalw-2.1 
! ./configure 2>&1 1>/dev/null
!make &> /dev/null
!make install &> /dev/null
%cd /content/
!rm clustalw-2.1.tar.gz

print('Dependencies created!')

In [ ]:
#@title BLAST local installation - OPTIONAL AND NOT RECOMMENDED... UNLESS YOU HAVE A LOT OF TIME #
#@markdown Only if you really want to use the local blast. This can take a long time!


print('This might take a few minutes... so take a coffe and relax!')

print('Installing BLAST...')
!wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.13.0+-src.tar.gz 
!mkdir blast 2>&1 1>/dev/null
!tar -xf ncbi-blast-2.13.0+-src.tar.gz 2>&1 1>/dev/null
!./ncbi-blast-2.13.0+-src/c++/configure 2>&1 1>/dev/null
!cd /content/ncbi-blast-2.13.0+-src/c++/ReleaseMT/build && /usr/bin/make all_r 2>&1 1>/dev/null

print('Creating the BLAST database...')
!wget https://ftp.ncbi.nlm.nih.gov/blast/db/swissprot.tar.gz 
!wget https://ftp.ncbi.nlm.nih.gov/blast/db/pdbaa.tar.gz 

!mkdir db 2>&1 1>/dev/null
!mkdir ./db/swissprot 2>&1 1>/dev/null
!mkdir ./db/pdbaa 2>&1 1>/dev/null
!tar -xf swissprot.tar.gz --directory ./db/swissprot 2>&1 1>/dev/null
!tar -xf pdbaa.tar.gz --directory ./db/pdbaa 2>&1 1>/dev/null


In [ ]:
#@title Import dependencies
#@markdown Import all necessary dependencies to run the CapiPy modules
import urllib.request
import urllib.parse
import glob
import os
import shutil
import numpy
import time
import urllib
import pandas as pd
import traceback
import requests
import csv
import re
import py3Dmol

from Bio.Blast import NCBIWWW
from Bio.Blast.Applications import NcbiblastpCommandline
from Bio import SeqIO, AlignIO, SearchIO, Align
from Bio.PDB import is_aa, PDBList, PDBIO, Superimposer
from Bio.PDB.Selection import unfold_entities
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import *
from modeller import *
from modeller.automodel import *
from sys import platform
from Bio import BiopythonWarning
from Bio.Blast import NCBIXML
from Bio.Align import substitution_matrices
from Bio.Align.Applications import ClustalwCommandline
from Bio import Entrez, Seq, SeqIO, SearchIO
clustalw_exe = 'clustalw2'

In [ ]:
#@title MODULE 1 - MODELLER
%cd /content
#@markdown Indicate your email address. This is for uniprot retreival and no data is saved.
EMAIL_ADDRESS = "davidrourap@gmail.com" #@param {type:"string"}
#@markdown Indicate the project name you want to give. A new folder will be created in /content/ with that name and will contain the run of all modules. 
PROJECT_NAME = "TEST" #@param {type:"string"}
#@markdown Enter the 1-letter amino acid code of your protein. Only amino acid letters are allowed. 
PROTEIN_SEQUENCE = "MDIGINSDPQTQDYQALDRAHHLHPFTDFKALGEEGSRVVTHAEGVYIHDSEGNRILDGMAGLWCVNLGYGRRELVEAATAQLEQLPYYNTFFKTTHPPAVRLAEKLCDLAPAHINRVFFTGSGSEANDTVLRMVRRYWALKGQPDKQWIIGRENAYHGSTLAGMSLGGMAPMHAQGGPCVPGIAHIRQPYWFGEGRDMSPEAFGQTCAEALEEKILELGEEKVAAFIAEPVQGAGGAIMPPESYWPAVKKVLAKYDILLVADEVICGFGRLGEWFGSQHYGLEPDLMPIAKGLSSGYLPIGGVLVGDRVAETLIEEGGEFFHGFTYSGHPTCAAVALKNLELLEAEGVVDRVRDDLGPYLAERWASLVDHPIVGEARSLGLMGALELVADKTTGQRFDKSLGAGNLCRDLCFANGLVMRSVGDTMIISPPLVIRREEIDELVELARRALDETARQLTQVPHTQEEPTAKLAAAENLYFQGLEHHHHHH" #@param {type:"string"}
#@markdown Indicate if you'd want to use the alphafold structure or a PDB structure to model your protein.
MODELLING = "ALPHAFOLD" #@param ["PDB", "ALPHAFOLD"]

#@markdown Ready to go! Press play. This can take up to 10 minutes.

initial_location = os.getcwd()
working_directory = PROJECT_NAME
if os.path.exists("./" + working_directory) is False:
   os.mkdir("./" + working_directory)
   print("Directory ", working_directory, " created.")
   os.chdir("./" + working_directory)
else:
   os.chdir("./" + working_directory)
if os.path.exists("./" + "Modeller") is False:
    os.mkdir("Modeller")

os.chdir("./Modeller")

print('Finding the best hit from UniProt...')
!python /content/ncbiblast.py --quiet --email=$EMAIL_ADDRESS --stype=protein --program=blastp --database=uniprotkb_refprotswissprot --outfile=uniprot_blast_result --sequence=$PROTEIN_SEQUENCE
upblastfiles = glob.glob('uniprot_blast*')
upblastfiles.remove('uniprot_blast_result.out.txt')
for file in upblastfiles:
    os.remove(file)
!mv uniprot_blast_result.out.txt uniprot_blast_result.txt  



def main(PROJECT_NAME, PROTEIN_SEQUENCE, MODELLING):
    protein_sequence = PROTEIN_SEQUENCE
    with open("query.fasta", "w+") as f:
        f.write(">query" + "\n" + protein_sequence)

    #Eliminate the his tag from the search to avoid partial hits with no real 
    #significance#

    if "HHHHH" in protein_sequence[0:20]:
        with open("protein_blast.fasta", "w+") as f:
            f.write(">query\n" + protein_sequence[20:])
    elif "HHHHH" in protein_sequence[len(protein_sequence) - 20:len(protein_sequence)]:
        with open("protein_blast.fasta", "w+") as f:
            f.write(">query\n" + protein_sequence[0:len(protein_sequence) - 20])
    else:
        with open("protein_blast.fasta", "w+") as f:
            f.write(">query" + "\n" + protein_sequence)

    # Identification of the best hit from the PDB database using query.fasta as query in blast
    template_modelling = MODELLING
    query = SeqIO.read(open("protein_blast.fasta"), format="fasta")
    print("Finding the best hit from the PDB...")
    pdb_blast = NCBIWWW.qblast("blastp",
                               "pdbaa",
                                query,
                                auto_format='XML')
    with open('pdb_blast_result.xml', 'w') as out_handle:
        out_handle.write(pdb_blast.read())
    pdb_blast.close()
   
    blast_pdb_result = SearchIO.read("pdb_blast_result.xml", "blast-xml") 
    if len(blast_pdb_result) > 0:
        blast_qresult = SearchIO.read("pdb_blast_result.xml", "blast-xml")[0]
        best_hit = blast_qresult[0]
        pdb_id = best_hit.hit.id.split("|")[1]
        print("Your blast returned the following as the best hit:\n" + str(pdb_id)
          + ".\n Visit https://www.rcsb.org/structure/" + str(pdb_id) + " for more information about the model.")


    else:
      print("Your blast doesn't seem to contain any result. Check the sequence and run the program again. "
          "If you run it over the server, consider using local blast. The program is going to quit now.")


    parser = PDBParser()
    ppb = PPBuilder()
    pdbl = PDBList()
    if MODELLING == "ALPHAFOLD":
        #Find and download the Alphafold prediction from Uniprot
        print('Downloading AlphaFold prediction for your protein...')       
        uniprot_blast = SearchIO.read('uniprot_blast_result.txt', 'blast-text')       
        if len(uniprot_blast) > 0:
            identifier = uniprot_blast[0].id.split(':')[1]
            urllib.request.urlretrieve('https://alphafold.ebi.ac.uk/files/AF-' + identifier + '-F1-model_v4.pdb',
                                  './' +identifier +'.pdb')
            print('Done! Your protein has the UniProt number: ' + str(identifier))
            
            model_pdbfile = identifier +'.pdb'
            model_file_name = str(identifier)
            model_structure = parser.get_structure(model_file_name, model_pdbfile)
            model_chain = model_structure.get_chains() 
        else:
            template_modelling = 'PDB'
    elif template_modelling == 'PDB':
        print('No hit was found in Uniprot... Changing to MODELLER.')
        pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb')
        model_pdbfile = 'pdb' + pdb_id.casefold() + '.ent'
        model_file_name = str(pdb_id)
        model_structure = parser.get_structure(model_file_name, model_pdbfile)
        model_chain = model_structure.get_chains()

    # Extract first hit and create a fasta with its pdb identifier and its sequence
    # Method for alignment with modeller. The option determines if only aa are used for the alignment or not.
    # This helps for enzymes with discontinuities and internal cofactors such as MIO dependant enzymes.
    def align_for_modeller(OPTION):
        for chain in model_structure.get_chains():
            last_chain = chain.id
            tnum_res = (len([_ for _ in chain.get_residues() if is_aa(_)]))
        for pp in ppb.build_peptides(model_structure[0][str(last_chain)], aa_only=OPTION):
            with open("model.fasta", "a") as f:
                sequence = pp.get_sequence()
                f.write(str(sequence))
        mdel = model_structure[0]
        chain = mdel[last_chain]
        res_list = unfold_entities(chain, "R")
        for residue in res_list:
            nid_1aa = (res_list[0].get_id())[1]
            nid_lastaa = (res_list[tnum_res - 1].get_id())[1]
        # Modify files so they have only 1 identifier + seq (id_1 or query)#
        with open('model.fasta', 'r') as o:
            data = o.read()
        with open('model.fasta', 'w') as mo:
            mo.write(">" + model_file_name + "\n" + data)
        with open("query.fasta") as o2:
            line2 = o2.readlines()
        line2[0] = ">query\n"
        with open("query.fasta", "w") as m1:
            m1.writelines(line2)
            # Combine original sequence and first hit into a fasta input file for the alignment#
        filenames = ["query.fasta", "model.fasta"]
        with open('alignment.fasta', 'w+') as aligninput:
            for files in filenames:
                with open(files) as infile:
                    aligninput.write(infile.read())
                aligninput.write("\n")
                # Profile-profile alignment using salign from modeller#
        log.none()
        aln = alignment(env, file='alignment.fasta', alignment_format='FASTA')
        aln.salign(rr_file='${LIB}/blosum62.sim.mat', gap_penalties_1d=(-500, 0), output='', align_block=15,
                    align_what='PROFILE', alignment_type='PAIRWISE', comparison_type='PSSM', similarity_flag=True,
                    substitution=True, smooth_prof_weight=10.0)
        aln.write(file='salign.ali', alignment_format='PIR')
        print("Alignment of template and query for modelling successfull.")
        # Fix formatting of the .ali file to specify the 1st and last aminoacid and the structure of the model protein#
        shutil.copyfile("salign.ali", "salign1.ali")
        with open("salign1.ali", "r") as file:
            filedata = file.read()
        replacement = filedata.replace(">P1;" + str(model_file_name) + "\nsequence::: :: :::-1.00:-1.00",
                                        ">P1;" + str(model_file_name) + "\nstructure" + ":" + str(model_file_name) + ":" + str(
                                            nid_1aa) + ":" + str(last_chain) + ":" + str(nid_lastaa) + ": ::::")
        with open("salign1.ali", "w+") as f:
            f.write(replacement)
        # Make a single model of the query sequence using: salign1.ali and model.pdb#
        a = automodel(env, alnfile="salign1.ali", knowns=model_file_name, sequence="query")
        a.starting_model = 1
        a.ending_model = 1
        a.make()
        # Check how good the alignment is#
        pir_alignment = AlignIO.read("salign1.ali", "pir")
        total_length = len(pir_alignment[0])
        gaps_1 = 0
        gaps_2 = 0
        for aas in pir_alignment[0].seq:
            if aas == "-":
                gaps_1 += 1
        for aas in pir_alignment[1].seq:
            if aas == "-":
                gaps_2 += 1
        if gaps_1 / total_length > 0.5 or gaps_2 / total_length > 0.5:
            print(
                "\nYour model protein covers less than half of the query. The created model could be inaccurate, " +
                "please check the result before continuing with anything else.\n")




    print('Creating your model with Modeller...')
    env = Environ()        
    # Run the previous method, changing the option if it fails the first time. If non of them work,
    # raise and error and quit the program.      
    try:
        align_for_modeller(True)
        print("First model created succesfully")

    except BaseException:
        try:
            delete_files = ["alignment.fasta", "salign.ali", "salign1.ali", "model.fasta"]
            for files in delete_files:
                if os.path.exists(files):
                    os.remove(files)
            align_for_modeller(False)
        except BaseException:
            print(traceback.format_exc())
            delete_files = ["salign.ali", "salign1.ali", "model.fasta"]
            for files in delete_files:
                if os.path.exists(files):
                    os.remove(files)
            print(
                "\nThere is problem with your model. It could be related to unnatural amino acids or problems in "
                "the residue numbering. Please, retry with a different PDB or try and fix the errors in the file."
                "\nThe program is now going to quit. Sorry!")


    # Method to superimpose the chains#
    def superimpose_chains(dict):
        checks = {}
        # Check if clustalw is available.
        for k_chain in toalign.keys():
            chainmodel = parser.get_structure("CH0" + toalign[k_chain], "chain" + toalign[k_chain] + ".pdb")
            chainquery = parser.get_structure("Q00" + str(k_chain), "query_aligned" + str(k_chain) + ".pdb")
            ppb = PPBuilder()
            # To maximise the accuracy of the superimposer and avoid problems with alignments of low quality instead
            # of using the whole protein to superimpose, two regions at the start and end will be used. To do so, first
            # it's necessary to identify two regions at the start and end where both there are no gaps, checking the
            # alignment for a minimum of 40 positions without "-" in reading frames of 5 aminoacids.
            frac_chainmodelseq = []
            chainmodelseq = ""
            for pp in ppb.build_peptides(chainmodel):
                frac_chainmodelseq.append(pp.get_sequence())
            for fractions in frac_chainmodelseq:
                chainmodelseq += fractions
            for pp in ppb.build_peptides(chainquery):
                chainqueryseq = pp.get_sequence()
            with open("initial" + str(k_chain) + ".fasta", "w") as input_file:
                input_file.write(
                    ">model_" + toalign[k_chain] + "\n" + str(chainmodelseq) + "\n>query_" + str(k_chain) + "\n" + str(
                        chainqueryseq))

            ex_clustal = ClustalwCommandline(clustalw_exe, infile="initial" + str(k_chain) + ".fasta")
            stdout, stderr = ex_clustal()
            alignment = AlignIO.read("initial" + str(k_chain) + ".aln", "clustal")
            length = len(alignment[0])
            # Identify the first region of 40 aa without gaps in the alignment
            i = 0
            posm = 0
            posq = 0
            posmf = 0
            posqf = 0
            for a in range(int(length / 5)):
                if "-" not in alignment[0][i:i + 40].seq and "-" not in alignment[1][i:i + 40]:
                    posq = i
                    posm = i
                else:
                    i += 5
            # Translate the alignment position to the real position in each sequence
            for aa in alignment[0].seq[0:i]:
                if aa == "-":
                    posm -= 1
            for aa in alignment[1].seq[0:i]:
                if aa == "-":
                    posq -= 1
            # Identify in the last quarter of the the alignment, a region of 40 aa without gaps
            k = length - (length // 4)
            while k < length - 40:
                if "-" not in alignment[0][k:k + 40].seq and "-" not in alignment[1][k:k + 40].seq:
                    posqf = k
                    posmf = k
                    break
                else:
                    k += 5
            if k + 40 > length:
                k = length - (length // 4)
                while k < length - 40:
                    if "-" not in alignment[0][k:k + 40].seq and "-" not in alignment[1][k:k + 40].seq:
                        posqf = k
                        posmf = k
                        break
                    else:
                        k += 2
            if posqf == 0:
                posqf = length - (length // 4)
                posmf = length - (length // 4)
            # Translate the alignment position to the real position in each sequence
            gapsmodel = 0
            for aa in alignment[0].seq[0:k]:
                if aa == "-":
                    gapsmodel += 1
                    posmf -= 1
            for aa in alignment[1].seq[0:k]:
                if aa == "-":
                    posqf -= 1
            # Check if the pdb file starts at amino acid 1 or not. If not, fix the range to be applied to the model.
            mdel_res_list = unfold_entities(chainmodel, "R")
            first_aa_model = mdel_res_list[0].get_id()[1]
            if first_aa_model != 1:
                posm = posm + first_aa_model
                posmf = posmf + first_aa_model

            discont = []
            for res in mdel_res_list:
                if is_aa(res) is True:
                    discont.append(res.get_id()[1])
                    last_aa_model = res.get_id()[1]
            a = 0
            for a in range(len(discont) - 1):
                if discont[a] < posmf:
                    if discont[a + 1] != discont[a] + 1:
                        posmf += 1
            # Define the ranges for the superimposer
            sup = Superimposer()
            ata_q1 = range(posq, posq + 40)
            ata_q2 = range(posqf, posqf + 40)
            ata_m1 = range(posm, posm + 40)
            ata_m2 = range(posmf, posmf + 40)
            # Create the list of the residues to be superimposed
            ref_atoms = []
            q_atoms = []
            # Add to the created lists the residues identified before
            try:
                for ref_chain in chainmodel[0]:
                    for ref_res in ref_chain:
                        if ref_res.get_id()[1] in ata_m1 or ref_res.get_id()[1] in ata_m2:
                            ref_atoms.append(ref_res['CA'])
                for q_chain in chainquery[0]:
                    for q_res in q_chain:
                        if q_res.get_id()[1] in ata_q1 or q_res.get_id()[1] in ata_q2:
                            q_atoms.append(q_res['CA'])
                    # Use the created lists to execute superimposer if they have the same number of atoms
                if len(ref_atoms) == len(q_atoms):
                    sup.set_atoms(ref_atoms, q_atoms)
                    sup.apply(chainquery)
                    io = PDBIO()
                    io.set_structure(chainquery)
                    # Refine the model until rms>5 or 20 iterations
                    print("\nRefining the model for chain " + str(toalign[k_chain]) + "...")
                    if numpy.abs(sup.rms) > 7.5:
                        checks[str(k_chain)] = 1
                        print("Careful, the superposition of the chains has an RMS of >5 (RMS = " + str(
                            (numpy.abs(sup.rms)))[0:4] + ").\n\n")
                        io.save("query_aligned" + str(k_chain) + ".pdb")

                    else:
                        print("Perfect for chain " + toalign[k_chain] + ".\n")
                        io.save("query_aligned" + str(k_chain) + ".pdb")

                else:
                    print("\nThere is some problem with chain " + str(
                        toalign[k_chain]) + ". Trying an alternate solution...")

                    # Identify the most complete chain
                    chain_lengths = []
                    for chain in ordered_structure.get_chains():
                        chain_lengths.append(len(chain))
                    complete_chain = toalign[chain_lengths.index(max(chain_lengths))]
                    good_model = parser.get_structure("CH0" + complete_chain, "chain" + complete_chain + ".pdb")
                    try:
                        # Move a copy of the most complete chain to the position of the failed chain
                        range_model = range(int(last_aa_model / 2), int((last_aa_model / 2) + 20))
                        fchain_atoms = []
                        goodm_atoms = []
                        for ref_chain in chainmodel[0]:
                            for ref_res in ref_chain:
                                if ref_res.get_id()[1] in range_model:
                                    fchain_atoms.append(ref_res['CA'])
                        for goodm_chain in good_model[0]:
                            for goodm_res in goodm_chain:
                                if goodm_res.get_id()[1] in range_model:
                                    goodm_atoms.append(goodm_res['CA'])
                        sup.set_atoms(fchain_atoms, goodm_atoms)
                        sup.apply(good_model)
                        io = PDBIO()
                        io.set_structure(good_model)
                        io.save("good_model" + toalign[k_chain] + ".pdb")
                        # redefine the chainmodel to use the new superimposed file
                        chainmodel_fixed = parser.get_structure("CH0M", "good_model" + toalign[k_chain] + ".pdb")
                        ref_atomsfixed = []
                        q_atoms = []
                        for m_chain in chainmodel_fixed[0]:
                            for m_res in m_chain:
                                if m_res.get_id()[1] in range_model:
                                    ref_atomsfixed.append(m_res['CA'])
                        for q_chain in chainquery[0]:
                            for q_res in q_chain:
                                if q_res.get_id()[1] in range_model:
                                    q_atoms.append(q_res['CA'])
                        # Superimpose query with the new structure
                        sup.set_atoms(ref_atomsfixed, q_atoms)
                        sup.apply(chainquery)
                        io.set_structure(chainquery)
                        print("Superimposing the query to a fixed chain " + str(toalign[k_chain]) + "...")
                        io.save("query_aligned" + str(k_chain) + ".pdb")

                        count = 0
                        if numpy.abs(sup.rms) > 7.5:
                            checks[str(k_chain)] = 1
                            print("Careful, the superposition of the chains has an RMS of >7.5 (RMS = " + str(
                                (numpy.abs(sup.rms)))[0:4] + ").\n")

                        else:
                            print("Perfect for chain " + toalign[k_chain] + ".")

                    except BaseException as error:
                        checks[str(k_chain)] = 2
                        print("Didn't work either. Check the PDB file for discontinuties or errors.", error)
            except KeyError:
                checks[str(k_chain)] = 2
                print("Superimposition for chain " + str(toalign[k_chain]) + " did not work. "
                      "It could be that it's different from the initally modelled monomer.\n")
                break
        return checks

    # Check if the model protein is formed by  more than one chain. Although not prove of it being multimeric,
    # better more work than overwatch this. If so, ask user if a multimeric model using the template has to be created#
    print('Creating your model with Modeller...')
    env = Environ()
    pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb')
    model_pdbfile = 'pdb' + pdb_id.casefold() + '.ent'
    model_file_name = str(pdb_id)
    pdb_model_structure = parser.get_structure(model_file_name, model_pdbfile)
    model_chain = model_structure.get_chains() 
      
    chainid = []
    for chains in pdb_model_structure.get_chains():
        if chains.get_id() not in chainid:
            chainid.append(chains.get_id())

    number_of_chains = len(chainid)

    if number_of_chains > 1:
        for i in range(number_of_chains):
            query_id =  "query_aligned" + str(i) + ".pdb"
            shutil.copyfile("query.B99990001.pdb", query_id)
            mdl = Model(env, file=query_id)
            mdl.rename_segments(segment_ids=[chainid[i]])
            mdl.write(file=query_id)

        # split the chains avoiding any error if atoms disordered
        # - bug in biopython (https://www.biostars.org/p/380566/ and https://github.com/biopython/biopython/issues/455),
        # changed the following def in the PDB python file of biopython. This converst disordered residues into ordered
        # ones by eliminating all locations apart from the first one.#
        #############################################################################################
        # def get_unpacked_list(self):                                                               #
        #     """                                                                                   #
        #     Returns all atoms from the residue,                                                   #
        #     in case of disordered, keep only first alt loc and remove the alt-loc tag             #
        #     """                                                                                   #
        #     atom_list = self.get_list()                                                           #
        #     undisordered_atom_list = []                                                           #
        #     for atom in atom_list:                                                                #
        #         if atom.is_disordered():                                                          #
        #         if atom.is_disordered():                                                          #
        #             atom.altloc=" "                                                               #
        #             undisordered_atom_list.append(atom)                                           #
        #         else:                                                                             #
        #             undisordered_atom_list.append(atom)                                           #
        #     return undisordered_atom_list                                                         #
        #############################################################################################
        io = PDBIO()
        io.set_structure(pdb_model_structure)
        io.save("ord_pdb" + pdb_id.casefold() + ".ent")
        ordered_pdbfile = "ord_pdb" + pdb_id.casefold() + ".ent"
        file = pdb_id.casefold()
        ordered_structure = parser.get_structure(file, ordered_pdbfile)
        k = 1
        for chain in ordered_structure.get_chains():
            for k in range(number_of_chains):
                chain = ordered_structure[0][chainid[k]]
                io.set_structure(chain)
                io.save("chain" + str(chainid[k]) + ".pdb")
                k += 1

        # Use the method defined to superimpose the chains.
        toalign = {}
        ch = 0
        for chains in chainid:
            if ch < number_of_chains:
                toalign[ch] = chainid[ch]
                ch += 1
        checks = superimpose_chains(toalign)
        
        # Add the files together to create a query_template with all subunits
        filenames = glob.glob("query_aligned*.pdb")
        if 2 not in checks.values():
            with open('query_multimeric.pdb', 'w+') as fullquerymodel:
                for names in filenames:
                    with open(names) as infile:
                        for line in infile:
                            if 'END' not in line:
                                fullquerymodel.write(line)
                fullquerymodel.write("END\n")
            if 1 in checks.values():
                print("While creating your multimeric model some errors occured. You can find your model in " + str(
                    os.getcwd()) + " in a file named query_multimeric.pdb. "
                                  "BUT CHECK IT BEFORE USING IT IN ANY FURTHER STEP!")
            else:
                print(
                    "\tFinished running the module Blast&Modeller module!\nCreation of your model was succesfull. "
                    "You can find it in " + str(os.getcwd()) + " in a file named query_multimeric.pdb along with the "
                    "monomeric modelled structure (query_monomeric.pdb), the sequences in fasta format, the "
                    "blast result and the alignment. You can continue now running the ActiveSiteID!")
    else:
        mdl = Model(env, file="query.B99990001.pdb")
        mdl.rename_segments(segment_ids="A")
        mdl.write(file="query.B99990001.pdb")
        print("\tFinished running the module Blast&Modeller module!\n You can find your result files in " + str(
            working_directory) + " in the Blast&Modeller folder.\nIf you think your protein should have a multimeric " +
            "assembly, check https://www.rcsb.org/structure/" + str(pdb_id) +
            " for more information. You can continue now running the ActiveSiteID!")

    # Clean the directory of intermediate files which are not necessary
    try:
        os.rmdir("obsolete")
    except FileNotFoundError:
        pass
    os.rename("query.B99990001.pdb", "query_monomer.pdb")
    if os.path.isfile('salign1.ali') == True:
        os.rename("salign1.ali", "alignment.ali")
    os.rename(model_pdbfile, 'pdb_template.pdb')
    os.rename('./' +identifier +'.pdb', 'alphafold_template.pdb')
    keep = ["query_multimeric.pdb", "query_monomer.pdb", "blast_result.xml", 'uniprot_blast_result.xml',
            'pdb_template.pdb', "query.fasta", "model.fasta", ]
    files = []
    for f in os.listdir("."):
        if os.path.isfile:
            files.append(f)

    for f in files:
        if f not in keep:
            try:
              os.remove(f)
            except BaseException:
              pass

    os.chdir(initial_location)  

if __name__ == '__main__':
  main(PROJECT_NAME, PROTEIN_SEQUENCE, MODELLING)

In [ ]:
#@markdown View generated model (red) and the template (blue).

#@markdown Indicate the Modeller folder where the files are contained. Ex. /content/Test/Modeller
MODELLER_FOLDER = "/content/TEST/Modeller" #@param {type:"string"}

os.chdir(MODELLER_FOLDER)
os.getcwd()

import py3Dmol
view = py3Dmol.view()
view.addModel(open('pdb_template.pdb','r').read(),'pdb')
try:
    view.addModel(open('query_multimeric.pdb', 'r').read(),'pdb')
except FileNotFoundError:
    view.addModel(open('query_monomeric.pdb', 'r').read(),'pdb')

view.setStyle({'model': -1}, {"cartoon": {'color': 'red'}})
view.setStyle({'model': 0}, {"cartoon": {'color': 'blue'}})
view.zoomTo()
view.show()

In [ ]:
#@title MODULE 1.1 - QUATERNARY STRUCTURE
#@markdown Quaternary structure mimic from a template pdb file.

#@markdown Indicate the PDB id of the protein you want to mimic the quaternary structure to.
PDB_FILE = "4A6T" #@param {type:"string"}
#@markdown Indicate the full path to your monomeric model. Ex.
MONOMERIC_STRUCTURE = "/content/TEST/Modeller/query_monomer.pdb" #@param {type:"string"}

warnings.simplefilter('ignore', BiopythonWarning)

working_directory = os.path.abspath((os.path.join(MONOMERIC_STRUCTURE, os.pardir)))
%cd $working_directory
keep = os.listdir('.')

def superimpose_chains(dict):
    checks = {}
    # Check if clustalw is available.
    for k_chain in toalign.keys():
        chainmodel = parser.get_structure("CH0" + toalign[k_chain], "chain" + toalign[k_chain] + ".pdb")
        chainquery = parser.get_structure("Q00" + str(k_chain), "query_aligned" + str(k_chain) + ".pdb")
        ppb = PPBuilder()
        # To maximise the accuracy of the superimposer and avoid problems with alignments of low quality instead
        # of using the whole protein to superimpose, two regions at the start and end will be used. To do so, first
        # it's necessary to identify two regions at the start and end where both there are no gaps, checking the
        # alignment for a minimum of 40 positions without "-" in reading frames of 5 aminoacids.
        frac_chainmodelseq = []
        chainmodelseq = ""
        for pp in ppb.build_peptides(chainmodel):
            frac_chainmodelseq.append(pp.get_sequence())
        for fractions in frac_chainmodelseq:
            chainmodelseq += fractions
        for pp in ppb.build_peptides(chainquery):
            chainqueryseq = pp.get_sequence()
        with open("initial" + str(k_chain) + ".fasta", "w") as input_file:
            input_file.write(
                ">model_" + toalign[k_chain] + "\n" + str(chainmodelseq) + "\n>query_" + str(k_chain) + "\n" + str(
                    chainqueryseq))

        ex_clustal = ClustalwCommandline(clustalw_exe, infile="initial" + str(k_chain) + ".fasta")
        stdout, stderr = ex_clustal()
        alignment = AlignIO.read("initial" + str(k_chain) + ".aln", "clustal")
        length = len(alignment[0])
        # Identify the first region of 40 aa without gaps in the alignment
        i = 0
        posm = 0
        posq = 0
        posmf = 0
        posqf = 0
        for a in range(int(length / 5)):
            if "-" not in alignment[0][i:i + 40].seq and "-" not in alignment[1][i:i + 40]:
                posq = i
                posm = i
            else:
                i += 5
        # Translate the alignment position to the real position in each sequence
        for aa in alignment[0].seq[0:i]:
            if aa == "-":
                posm -= 1
        for aa in alignment[1].seq[0:i]:
            if aa == "-":
                posq -= 1
        # Identify in the last quarter of the the alignment, a region of 40 aa without gaps
        k = length - (length // 4)
        while k < length - 40:
            if "-" not in alignment[0][k:k + 40].seq and "-" not in alignment[1][k:k + 40].seq:
                posqf = k
                posmf = k
                break
            else:
                k += 5
        if k + 40 > length:
            k = length - (length // 4)
            while k < length - 40:
                if "-" not in alignment[0][k:k + 40].seq and "-" not in alignment[1][k:k + 40].seq:
                    posqf = k
                    posmf = k
                    break
                else:
                    k += 2
        if posqf == 0:
            posqf = length - (length // 4)
            posmf = length - (length // 4)
        # Translate the alignment position to the real position in each sequence
        gapsmodel = 0
        for aa in alignment[0].seq[0:k]:
            if aa == "-":
                gapsmodel += 1
                posmf -= 1
        for aa in alignment[1].seq[0:k]:
            if aa == "-":
                posqf -= 1
        # Check if the pdb file starts at amino acid 1 or not. If not, fix the range to be applied to the model.
        mdel_res_list = unfold_entities(chainmodel, "R")
        first_aa_model = mdel_res_list[0].get_id()[1]
        if first_aa_model != 1:
            posm = posm + first_aa_model
            posmf = posmf + first_aa_model

        discont = []
        for res in mdel_res_list:
            if is_aa(res) is True:
                discont.append(res.get_id()[1])
                last_aa_model = res.get_id()[1]
        a = 0
        for a in range(len(discont) - 1):
            if discont[a] < posmf:
                if discont[a + 1] != discont[a] + 1:
                    posmf += 1
        # Define the ranges for the superimposer
        sup = Superimposer()
        ata_q1 = range(posq, posq + 40)
        ata_q2 = range(posqf, posqf + 40)
        ata_m1 = range(posm, posm + 40)
        ata_m2 = range(posmf, posmf + 40)
        # Create the list of the residues to be superimposed
        ref_atoms = []
        q_atoms = []
        # Add to the created lists the residues identified before
        try:
            for ref_chain in chainmodel[0]:
                for ref_res in ref_chain:
                    if ref_res.get_id()[1] in ata_m1 or ref_res.get_id()[1] in ata_m2:
                        ref_atoms.append(ref_res['CA'])
            for q_chain in chainquery[0]:
                for q_res in q_chain:
                    if q_res.get_id()[1] in ata_q1 or q_res.get_id()[1] in ata_q2:
                        q_atoms.append(q_res['CA'])
                # Use the created lists to execute superimposer if they have the same number of atoms
            if len(ref_atoms) == len(q_atoms):
                sup.set_atoms(ref_atoms, q_atoms)
                sup.apply(chainquery)
                io = PDBIO()
                io.set_structure(chainquery)
                # Refine the model until rms>5 or 20 iterations
                print("\nRefining the model for chain " + str(toalign[k_chain]) + "...")
                if numpy.abs(sup.rms) > 7.5:
                    checks[str(k_chain)] = 1
                    print("Careful, the superposition of the chains has an RMS of >5 (RMS = " + str(
                        (numpy.abs(sup.rms)))[0:4] + ").\n\n")
                    io.save("query_aligned" + str(k_chain) + ".pdb")

                else:
                    print("Perfect for chain " + toalign[k_chain] + ".\n")
                    io.save("query_aligned" + str(k_chain) + ".pdb")

            else:
                print("\nThere is some problem with chain " + str(
                    toalign[k_chain]) + ". Trying an alternate solution...")

                # Identify the most complete chain
                chain_lengths = []
                for chain in ordered_structure.get_chains():
                    chain_lengths.append(len(chain))
                complete_chain = toalign[chain_lengths.index(max(chain_lengths))]
                good_model = parser.get_structure("CH0" + complete_chain, "chain" + complete_chain + ".pdb")
                try:
                    # Move a copy of the most complete chain to the position of the failed chain
                    range_model = range(int(last_aa_model / 2), int((last_aa_model / 2) + 20))
                    fchain_atoms = []
                    goodm_atoms = []
                    for ref_chain in chainmodel[0]:
                        for ref_res in ref_chain:
                            if ref_res.get_id()[1] in range_model:
                                fchain_atoms.append(ref_res['CA'])
                    for goodm_chain in good_model[0]:
                        for goodm_res in goodm_chain:
                            if goodm_res.get_id()[1] in range_model:
                                goodm_atoms.append(goodm_res['CA'])
                    sup.set_atoms(fchain_atoms, goodm_atoms)
                    sup.apply(good_model)
                    io = PDBIO()
                    io.set_structure(good_model)
                    io.save("good_model" + toalign[k_chain] + ".pdb")
                    # redefine the chainmodel to use the new superimposed file
                    chainmodel_fixed = parser.get_structure("CH0M", "good_model" + toalign[k_chain] + ".pdb")
                    ref_atomsfixed = []
                    q_atoms = []
                    for m_chain in chainmodel_fixed[0]:
                        for m_res in m_chain:
                            if m_res.get_id()[1] in range_model:
                                ref_atomsfixed.append(m_res['CA'])
                    for q_chain in chainquery[0]:
                        for q_res in q_chain:
                            if q_res.get_id()[1] in range_model:
                                q_atoms.append(q_res['CA'])
                    # Superimpose query with the new structure
                    sup.set_atoms(ref_atomsfixed, q_atoms)
                    sup.apply(chainquery)
                    io.set_structure(chainquery)
                    print("Superimposing the query to a fixed chain " + str(toalign[k_chain]) + "...")
                    io.save("query_aligned" + str(k_chain) + ".pdb")

                    count = 0
                    if numpy.abs(sup.rms) > 7.5:
                        checks[str(k_chain)] = 1
                        print("Careful, the superposition of the chains has an RMS of >7.5 (RMS = " + str(
                            (numpy.abs(sup.rms)))[0:4] + ").\n")

                    else:
                        print("Perfect for chain " + toalign[k_chain] + ".")

                except BaseException as error:
                    checks[str(k_chain)] = 2
                    print("Didn't work either. Check the PDB file for discontinuties or errors.", error)
        except KeyError:
            checks[str(k_chain)] = 2
            print("Superimposition for chain " + str(toalign[k_chain]) + " did not work. "
                  "It could be that it's different from the initally modelled monomer.\n")
            break
    return checks

try:
    shutil.copy(MONOMERIC_STRUCTURE, "./monomeric_structure.pdb")
except OSError:
    print("Monomeric model not found. Please try to answer again.")
answer_pdb_id = PDB_FILE
if len(answer_pdb_id) == 4:
    if answer_pdb_id.isalpha() is True:
        print("It seems that your format is not a PDBid. Try again.")
    elif answer_pdb_id.isnumeric() is True:
        print("Remember that PDBid are formed by letters and numbers.")
    else:
        id_1 = answer_pdb_id
        print(str(id_1) + " is going to be used as the template for your model.")
else:
    print("That doesn't seem to be an accepted format.")

urllib.request.urlretrieve("https://files.rcsb.org/download/" + id_1 + ".pdb1", './' + id_1 + '.pdb')
parser = PDBParser()
ppb = PPBuilder()
pdbfile = id_1 + '.pdb'
file_name = str(id_1.casefold())
structure = parser.get_structure(PDB_FILE, pdbfile)
env = Environ()

chainid = []
for chains in structure.get_chains():
    if chains.get_id() not in chainid:
        chainid.append(chains.get_id())

number_of_chains = len(chainid)

if number_of_chains > 1:
    i = 0
    for i in range(number_of_chains):
        shutil.copyfile("monomeric_structure.pdb", "query_aligned" + str(i) + ".pdb")
    # Give the newly created files a chain id.#
    chainid = []
    for chains in structure.get_chains():
        chainid.append(chains.get_id())
    chainrange = range(0, len(chainid))
    filenames = []
    for files in glob.glob("query_aligned*.pdb", recursive=True):
        filenames.append(files)
    i = 0
    for files in filenames:
        while i < len(filenames):
            mdl = Model(env, file=filenames[i])
            mdl.rename_segments(segment_ids=[chainid[i]])
            mdl.write(file="query_aligned"+str(i)+".pdb")
            i += 1

io = PDBIO()
io.set_structure(structure)
io.save("ord_pdb" + id_1.casefold() + ".ent")
ordered_pdbfile = "ord_pdb" + id_1.casefold() + ".ent"
file = id_1.casefold()
ordered_structure = parser.get_structure(file, ordered_pdbfile)
k = 0
for chain in ordered_structure.get_chains():
    for k in range(number_of_chains):
        chain = ordered_structure[0][chainid[k]]
        io.set_structure(chain)
        io.save("chain" + str(chainid[k]) + ".pdb")
        k += 1

# Use the method defined to superimpose the chains.
toalign = {}
ch = 0
for chains in chainid:
    if ch < number_of_chains:
        toalign[ch] = chainid[ch]
        ch += 1
        
checks = superimpose_chains(toalign)
filenames = glob.glob("query_aligned*.pdb")
if 2 not in checks.values():
    if os.path.isfile("query_multimeric") == True:
        with open('quat_multimeric.pdb', 'w+') as fullquerymodel:  
            for names in filenames: 
                with open(names) as infile: 
                    for line in infile:
                        if 'END' not in line:
                            fullquerymodel.write(line)
            fullquerymodel.write("END\n")
    else:
        with open('quat_as_' + answer_pdb_id + '.pdb', 'w+') as fullquerymodel:  
            for names in filenames: 
                with open(names) as infile: 
                    for line in infile:
                        if 'END' not in line:
                            fullquerymodel.write(line)
            fullquerymodel.write("END\n")
    if 1 in checks.values():
        print("While creating your multimeric model some errors occured. You can find your model in "
              + str(os.getcwd()) + " in a file named query_multimeric.pdb. BUT CHECK IT BEFORE USING IT IN ANY "
                                    "FURTHER STEP!")
    else:
        print("\tFinished running the module Blast&Modeller module!\nCreation of your model was succesfull."
              "You can find it in " + str(os.getcwd()) + " in a file named query_multimeric.pdb along with "
              "the monomeric modelled structure (query_monomeric.pdb), the sequences in fasta format, "
              "the blast result and the alignment. You can continue now running the ActiveSiteID!")
else:
    print("No multimeric assembly could be modelled due to errors in the chain or too high RMS. "
          "Please check the pdb file for discontinuities or related problems "
          "(check https://www.rcsb.org/structure/" + str(id_1) + "for more information). "
          "You could try to rerun the stand alone multimeric builder with another PDB id.")
   
keep.append(id_1 + '.pdb')
keep.append('quat_as_' + answer_pdb_id + '.pdb')

files = []
for f in os.listdir("."):
    if os.path.isfile:
        files.append(f)

for f in files:
    if f not in keep:
        os.remove(f)

In [ ]:
#@markdown View generated model (red) and the template (blue).

view = py3Dmol.view()
files = glob.glob('quat_as_*')
for pdb in files:
  original_pdb = pdb.split('_')[2]

view.addModel(open(original_pdb,'r').read(),'pdb')

view.addModel(open(files[0], 'r').read(),'pdb')

view.setStyle({'model': -1}, {"cartoon": {'color': 'red'}})
view.setStyle({'model': 0}, {"cartoon": {'color': 'blue'}})
view.zoomTo()

In [ ]:
#@title MODULE 2 - ACTIVE SITE ID
#@markdown Identify the active site of your protein by comparing it to the MS-CSA (https://www.ebi.ac.uk/thornton-srv/m-csa/)

#@markdown Indicate if you have run before the MODELLER module in the same project.
MODELLER_RUN = True #@param {type:"boolean"}
#@markdown Indicate the full path to your query protein sequence in fasta format. This can be in the Modeller directory (Ex. /content/test/query.fasta )
SEQUENCE_FASTA = "/content/TEST/Modeller/query.fasta" #@param {type:"string"}
#@markdown Choose if you want to directly view the highlited position of your active site.
SHOW_STRUCTURE = True #@param {type:"boolean"}

if MODELLER_RUN == True:
  modeller_directory = os.path.abspath((os.path.join(SEQUENCE_FASTA, os.pardir)))
  working_directory = os.path.abspath((os.path.join(modeller_directory, os.pardir)))
else:
  working_directory = os.getcwd()

%cd $working_directory
if os.path.exists("ActiveSite") is False:
    os.mkdir("ActiveSite")

os.chdir("./ActiveSite")

url_atlas = "https://www.ebi.ac.uk/thornton-srv/m-csa/media/flat_files/curated_data.csv"
print("Downloading the curated data from the Mechanism and Catalytic Site atlas - www.ebi.ac.uk -.")
with urllib.request.urlopen(url_atlas) as data, open('./curated_data.csv', 'w', encoding="utf-8") as f:
    f.write(data.read().decode())

try:
  shutil.copy(SEQUENCE_FASTA, "./query.fasta")
except FileNotFoundError:
  print('Make sure your SEQUENCE_FASTA is the full path to your file!')
  quit()

if MODELLER_RUN == True:
    with open(modeller_directory + "/model.fasta") as file:
        id_1 = file.readline().replace(">", "").rstrip().upper()
elif MODELLER_RUN == False:
    query = SeqIO.read(SEQUENCE_FASTA, "fasta")
    pdb_blast = NCBIWWW.qblast("blastp",
                               "pdbaa",
                                query,
                                auto_format='XML')
    with open('pdb_blast_result.xml', 'w') as out_handle:
        out_handle.write(pdb_blast.read())
    
    pdb_blast.close()   
    blast_pdb_result = SearchIO.read("pdb_blast_result.xml", "blast-xml") 
    if len(blast_pdb_result) > 0:
        blast_qresult = SearchIO.read("pdb_blast_result.xml", "blast-xml")[0]
        best_hit = blast_qresult[0]
        id_1 = best_hit.hit.id.split("|")[1]
    else:
        print('There seems to be no good homologue in the PDB... Sorry!')
        quit()

print("Searching for the uniprot equivalent of your pdb file...")
url = 'https://rest.uniprot.org/idmapping/run'
params = {'from': 'PDB', 'to': 'UniProtKB', 'ids': id_1}
headers = {'user_agent': 'Mozilla/5.0'}

try:       
    pdb_id = id_1
    handle = requests.get(
        'https://www.ebi.ac.uk/pdbe/api/mappings/' + pdb_id)
    handle_json = handle.json()
    for key, item in handle_json[pdb_id.lower()].items():
        if key == 'UniProt':
            uniprotID = list(item.keys())[0]
            print('Found the uniprot equivalency: ' + uniprotID)
except BaseException:
    Entrez.email = 'davidrourap@gmail.com'
    handle_random_id = Entrez.esearch(
        db="protein", term=id_1)
    record_random_id = Entrez.read(handle_random_id)
    entrez_id = record_random_id['IdList'][0]

    with Entrez.efetch(db="protein", rettype="gb", retmode="text", id=entrez_id) as handle:
        seq_record = SeqIO.read(handle, "gb")

    GB_id = seq_record.id
    fullURL = ('http://rest.uniprot.org/uniprot/search?'
                'query=xref:' + str(GB_id) + '&format=list')
    result = requests.get(fullURL)
    uniprotID = str(result.text).replace('\n', '')
    print('Found the uniprot equivalency: ' + uniprotID)
if uniprotID.isalpha():
    print("Your pdb does not appear to have a UniProt equivalent. Sorry!")
else:
    print("Uniprot id found. Extracting information...")

handle = urllib.request.urlopen("https://www.uniprot.org/uniprot/"+str(uniprotID)+".xml")
record = SeqIO.read(handle, "uniprot-xml")

protein_type = ""
proteinECnumber = ""

for information in record:
    if "submittedName_fullName" in record.annotations:
        protein_type = record.annotations["submittedName_fullName"]
        break
    elif "recommendedName_fullName" in record.annotations:
        protein_type = str(record.annotations["recommendedName_fullName"])
        break

for information in record:
    if "submittedName_ecNumber" in record.annotations:
        proteinECnumber = record.annotations['submittedName_ecNumber']
        break
    elif "recommendedName_ecNumber" in record.annotations:
        proteinECnumber = record.annotations['recommendedName_ecNumber']
        break

if isinstance(proteinECnumber, list):
    proteinECnumber = str(proteinECnumber[0])

if proteinECnumber == "":
    print("It seems the information provided by uniprot is not enough. "
          "Please, check https://www.uniprot.org/uniprot/"+str(uniprotID))
    print("Your protein doesn't seem to have an EC number... Not much we can do without it!")
    if proteinECnumber.upper().replace(" ", "") == "NO":
        print("Sorry, but without the EC number there is little we can do...")
        time.sleep(3)
        quit()


res = open("Active_site.txt", "a")
if isinstance(protein_type, list) is True:
    print("\nYour model protein, " + str(id_1)+", has been identified as a " + protein_type[0] +
          " with EC number " + str(proteinECnumber) + ".\n")
    print("\nYour model protein, " + str(id_1)+", has been identified as a " + protein_type[0] +
          " with EC number " + str(proteinECnumber) + ".\n", file=res)
elif isinstance(protein_type, str) is True:
    print("\nYour model protein, " + str(id_1)+", has been identified as a "
          + protein_type.replace("[", "").replace("]", "") + " with EC number " + str(proteinECnumber) + ".\n")
    print("\nYour model protein, " + str(id_1)+", has been identified as a "
          + protein_type.replace("[", "").replace("]", "") + " with EC number " + str(proteinECnumber) + ".\n",
          file=res)
else:
    print("\nYour model protein, " + str(id_1)+", has EC number " + str(proteinECnumber) + ".\n")
    print("\nYour model protein, " + str(id_1)+", has EC number " + str(proteinECnumber) + ".\n", file=res)
res.close()
time.sleep(3)

# In a few cases, uniprot provides already the information about the catalytic site. In this case, we will compare
# the pdb file provided to the query and avoid using the ATLAS site.
act_site = []
bind_site = []
catalytic_res = {}
cofactor = []

# Find if information about the catalytic residues exists in the uniprot xml
for i in range(len(record.features)):
    if record.features[i].type == "active site":
        act_site.append(int(record.features[i].location.end)-1)
    elif record.features[i].type == "binding site":
        bind_site.append(int(record.features[i].location.end)-1)

cofactorslist = ["NAD", "FAD", "FMN", "SAM", "PLP", "ATP", "UTP", "ADP", "UDP", "CTP", "PAPS", "acetyl COA", "Zn",
                 "Fe", "Cu", "K", "Mg", "Mo", "Ni", "Se"]
if "comment_cofactor" in record.annotations:
    for ncof in cofactorslist:
        for act in range(len(record.annotations["comment_cofactor"])):
            if ncof in record.annotations["comment_cofactor"][act]:
                cofactor.append(ncof)

# If it exists, extract that information and ask for user confirmation to use it
if len(act_site) > 0 and use_uniprot == 'YES':
    up_maxid = record.annotations["accessions"][0]
    for i in range(len(act_site)):
        catalytic_res[record.seq[act_site[i]]] = act_site[i]
    copyfile("query.fasta", "refsequp.fasta")
    with open("refsequp.fasta", "a") as f:
        f.write("\n>" + str(record.annotations["accessions"][0])+"\n" + str(record.seq))

if len(act_site) == 0 or use_uniprot == "NO":
    print("Your model protein does not have an active site identified in UniProt. "
          "Proceeding to check in the M-CSA database...")
    time.sleep(3)
    # Create dictionary from database with uniprot id and EC number only of those
    # sharing the same ECnumber with the model#
    upidEC = {}
    curated_data = csv.reader(open("./curated_data.csv", "r"))
    for row in curated_data:
        upidEC[row[3]] = str(row[1])
    searchUP_dict = {}
    while len(searchUP_dict) == 0:
        for key, value in upidEC.items():
            if proteinECnumber in key:
                searchUP_dict[key] = value
        else:
            proteinECnumberlist = proteinECnumber.split(".")
            maxi = len(proteinECnumberlist)
            proteinECnumber = str(proteinECnumberlist[0]) + "." + str(proteinECnumberlist[1])
            if proteinECnumber in key:
                searchUP_dict[key] = value

    # Clean uniprot ids if they have more than 6 characters
    uniprotids = list(searchUP_dict.values())
    uniprotids_clean = []
    for i in uniprotids:
        if len(i) != 6:
            uniprotids_clean.append(i[0:6])
        else:
            uniprotids_clean.append(i)
    # Retrieve sequences of those uniprot identifiers from the website
    upECrefseq = []
    i = 0
    for x in uniprotids_clean:
        handle2 = urllib.request.urlopen("https://www.uniprot.org/uniprot/" + uniprotids_clean[i] + ".xml")
        record2 = SeqIO.read(handle2, "uniprot-xml")
        upECrefseq.append(record2.seq)
        i += 1
    # Use Pairwise alignment to identify the best hit from the M-CSA list
    # Create the fasta file for the identified hit sin the M-CSA
    i = 0
    for seqs in uniprotids_clean:
        with open(uniprotids_clean[i]+".fasta", "w") as f:
            f.write("\n>"+uniprotids_clean[i]+"\n" + str(upECrefseq[i]))
        i += 1

    # Pairwise alignment execution. The options used are the typical from the BLOSUM62 substitution matrix

    aligner = Align.PairwiseAligner()
    alphabet = "PROTEIN"
    aligner.open_gap_score = -10
    aligner.extend_gap_score = -0.1
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

    # Save into a list the scores of each alignment

    alignment_scores = []
    i = 0
    for seqs in uniprotids_clean:
        seq1 = SeqIO.read("query.fasta", "fasta")
        seq2 = SeqIO.read(uniprotids_clean[i]+".fasta", "fasta")
        alignments = aligner.align(seq1.seq, seq2.seq)
        alignment_scores.append(alignments.score)
        i += 1

    # Idenfity the UniProt ID with the maximal identity to the query sequence

    try:
        up_maxid = uniprotids_clean[alignment_scores.index(max(alignment_scores))]
        print("The best hit for your protein is uniprotID: " + str(up_maxid) + ". \n")
        time.sleep(3)
    except BaseException:
        print("Could not find a protein with enough homology with your query. "
              "You can try again with a different model!")
        time.sleep(4)
        quit()
    time.sleep(2)

    # From the UniProtID with max identity, retrieve the catalytic residues annotated in the M-CSA csv file
    curated_data = csv.reader(open("./curated_data.csv", "r"))
    for row in curated_data:
        if str(row[1])[0:6] == str(up_maxid) and row[4] == "residue":
            if row[5] not in catalytic_res.keys():
                catalytic_res[row[5]] = [row[7]]
            elif row[5] in catalytic_res.keys() and row[7] not in catalytic_res[row[5]]:
                catalytic_res[row[5]].append(row[7])

    # From the UniProtID with max identity, retrieve the cofactors (if any)
    curated_data = csv.reader(open("./curated_data.csv", "r"))
    for row in curated_data:
        if str(row[1]) == str(up_maxid) and row[4] == "cofactor":
            cofactor.append(row[8] + "(" + row[5] + ")")
    cofactor = set(cofactor)

# Alignment of the query sequence with the best hit from UniProt to identify the catalytic residues in the query.
# Create a fata file containing the query sequence and the sequence of the best UniProtID
    shutil.copyfile("query.fasta", "refsequp.fasta")

    with open("refsequp.fasta", "a") as f:
        f.write("\n>"+up_maxid+"\n" + str(upECrefseq[alignment_scores.index(max(alignment_scores))]))

# Use ClustalW for the alignment
sequp_align = ClustalwCommandline(clustalw_exe, infile="refsequp.fasta", score="percent")
stdout, stderr = sequp_align()
with open("alignment_output.txt", "w+") as clustalscore:
    clustalscore.write(stdout)

# From the ClustalW results, check if the identity is sufficient to continue with the active site identification
with open("alignment_output.txt") as c:
    for line in c:
        if "Sequences (1:2)" in line:
            score = int(''.join(filter(str.isdigit, line)))
if len(str(score)) == 3:
    score = score - 120
elif len(str(score)) == 4:
    score = score - 1200
elif len(str(score)) == 5:
    score = 100

print("\t Percentage of identity = " + str(score) + "\n")
time.sleep(3)
if 40 > score > 15:
    identity = 1
elif score <= 15:
    identity = 2
else:
    identity = 0

time.sleep(3)
if identity == 0:
    highhomology = "YES"
elif identity == 1:
    highhomology = "YES"
else:
    highhomology = "NO"
    print("Your query sequence alignment resulted in less than 15% identity. "
          "Prediction of the active site would not be accurate using it.\n")
    time.sleep(3)

# From alignment, and the catalytic residues identified in the M-CSA hit or the UniProt,
# return positions in your protein#
res = open("Active_site.txt", "a")
sequp_alignment = AlignIO.read("refsequp.aln", "clustal")

if highhomology == "YES":
    # Check that the list is not empty
    aaposition_cat = []
    aaposition_cat1 = []
    if len(list(catalytic_res.keys())[0]) >= 1:
        aaname_cat = list(catalytic_res.keys())
        aaposition_cat = list(catalytic_res.values())
        for aaposition in aaposition_cat:
            if isinstance(aaposition, list):
                for aa in aaposition:
                    aaposition_cat1.append(aa)
            else:
                aaposition_cat1.append(aaposition)
    
        # Convert the identified residues to a dictionary with the information in a more readable format
        sequp_alignment = AlignIO.read("refsequp.aln", "clustal")
        threetoone = {"Ala": "A", "Arg": "R", "Asn": "N", "Asp": "D", "Cys": "C", "Glu": "E", "Gln": "Q",
                      "Gly": "G", "His": "H", "Ile": "I", "Leu": "L", "Lys": "K", "Met": "M", "Phe": "F",
                      "Pro": "P", "Ser": "S", "Thr": "T", "Trp": "W", "Tyr": "Y", "Val": "V"}
        catalytic_res1lett = {}
        i = 0
        conv3to1 = 0
        for keys in catalytic_res.keys():
            if keys in threetoone.keys():
                conv3to1 = 1
                catalytic_res1lett[threetoone[str(aaname_cat[i])]] = aaposition_cat[i]
                i += 1
        if conv3to1 == 1:
            aaname_cat = list(catalytic_res1lett.keys())
        else:
            for positions in catalytic_res.keys():
                catalytic_res1lett[positions] = [catalytic_res[positions]]  
    else:
        aaname_cat = list(catalytic_res.keys())
        aaposition_cat = list(catalytic_res.values())
        aaposition_cat1 = list(catalytic_res.values())
        for positions in catalytic_res.keys():
            catalytic_res1lett[positions] = [catalytic_res[positions]]
    # From the known catalytic positions, identify the corresponding positions in the alignment
    print("Identifying the predicted catalytic residues in your protein...\n")
    time.sleep(3)
    a = 0
    positions_tocheck = []
    for it in aaposition_cat1:
        positions_tocheck.append(int(it)-1)
        a += 1
    i = 0
    align_catpos = []
    for i in range(a):
        pos_cat = positions_tocheck[i]
        gap_count, gap_count_1, gap_count_2, gap_count_3 = (0, 0, 0, 0)
        for aa in sequp_alignment[1][:positions_tocheck[i]]:
            if aa == "-":
                gap_count += 1
        if gap_count != 0:
            for aa1 in sequp_alignment[1][pos_cat:pos_cat + gap_count]:   
                if aa == "-":
                    gap_count_1 += 1
            if gap_count_1 != 0:
                for aa2 in sequp_alignment[1][pos_cat:pos_cat + gap_count + gap_count_1]:   
                    if aa == "-":
                        gap_count_2 += 1
                if gap_count_2 != 0:
                    for aa3 in sequp_alignment[1][pos_cat:pos_cat + gap_count + gap_count_1 + gap_count_2]:   
                        if aa == "-":
                            gap_count_3 += 1
        align_catpos.append(pos_cat + gap_count + gap_count_1 + gap_count_2 + gap_count_3)
        i += 1
    # Extract in list from the dictionary of 1lettcode
    lettkeys = list(catalytic_res1lett.keys())

    # From the positions in the alignment, identify the real position in the query sequence
    query_catalytic_res = {}
    for lett in lettkeys:
        query_catalytic_res[lett] = []

    query = SeqIO.read("query.fasta", "fasta")
    for pos in align_catpos:
        pos_cat = pos
        pos_cat_real = pos_cat
        for aas in sequp_alignment[0][0:pos_cat+1]:
            if aas == "-":
                pos_cat_real -= 1
        try:
            if isinstance(query_catalytic_res[query[pos_cat_real]], list) is True:
                query_catalytic_res[query[pos_cat_real]].append(pos_cat_real+1)
            elif isinstance(query_catalytic_res[query[pos_cat_real]], str) is True:
                query_catalytic_res[query[pos_cat_real]] = [query_catalytic_res[query[pos_cat_real]]]
                query_catalytic_res[query[pos_cat_real]].append(pos_cat_real+1)
        except KeyError:
            try:
                if isinstance(query_catalytic_res[query[pos_cat_real+1]], list) is True:
                    query_catalytic_res[query[pos_cat_real+1]].append(pos_cat_real+2)
                elif isinstance(query_catalytic_res[query[pos_cat_real+1]], str) is True:
                    query_catalytic_res[query[pos_cat_real+1]] = [query_catalytic_res[query[pos_cat_real+1]]]
                    query_catalytic_res[query[pos_cat_real+1]].append(pos_cat_real+2)
            except KeyError:
                try:
                    if isinstance(query_catalytic_res[query[pos_cat_real-1]], list) is True:
                        query_catalytic_res[query[pos_cat_real-1]].append(pos_cat_real)
                    elif isinstance(query_catalytic_res[query[pos_cat_real-1]], str) is True:
                        query_catalytic_res[query[pos_cat_real-1]] = [query_catalytic_res[query[pos_cat_real-1]]]
                        query_catalytic_res[query[pos_cat_real-1]].append(pos_cat_real)
                except KeyError:
                    query_catalytic_res[query[pos_cat_real]] = str(pos_cat_real+1) + "*"    

    query_catalytic_resC = query_catalytic_res.copy()
    for key in query_catalytic_resC.keys():
        if len(query_catalytic_resC[key]) == 0:
            query_catalytic_res[key] = "Present in model and absent in the query."
    time.sleep(3)

    # Print the results both in the console and in a file
    print("The predicted active site is formed by:")
    print("The predicted active site is formed by:", file=res)
    for key, value in query_catalytic_res.items():
        print(str(key) + "--> " + str(value).replace("[", "").replace("]", ""))
        print(str(key) + "--> " + str(value).replace("[", "").replace("]", ""), file=res)
    print("\n* : Residue predicted in the query but not present as part of the active site of the model.\n")
    print("\n* : Residue predicted in the query but not present as part of the active site of the model.\n",
          file=res)
    time.sleep(3)
else:
    if len(bind_site) > 0:
        print("Due to low homology, calculation of the active site failed. "
              "Maybe you could check the model pdb file information in  "
              "https://www.uniprot.org/uniprot/"+str(uniprotID) + " for more information.")
        print("Due to low homology, calculation of the active site failed. "
              "Maybe you could check the model pdb file information in  "
              "https://www.uniprot.org/uniprot/"+str(uniprotID) + " for more information.", file=res)
        time.sleep(3)
    else:
        print("Due to low homology, calculation of the active site failed. "
              "Maybe you could check the model pdb file information in  "
              "https://www.uniprot.org/uniprot/"+str(uniprotID) + " for more information.")
        print("Due to low homology, calculation of the active site failed. "
              "Maybe you could check the model pdb file information in  "
              "https://www.uniprot.org/uniprot/"+str(uniprotID) + " for more information.", file=res)

# At last, from the EC number write some general information of the protein. If they are supposed to have a cofactor
# but could not be identified, print also that!
if len(cofactor) > 0:
    print("\nIdentified cofactor(s): ")
    print("\nIdentified cofactor(s): ", file=res)
    for x in cofactor:
        print(x)
        print(x, file=res)
elif len(bind_site) > 0:
    print("Also, in Uniprot some binding sites are indicated for the pdb model in positions: ")
    print(*bind_site, sep=",")
    print("Also, in Uniprot some binding sites are indicated for the pdb model in positions: ", file=res)
    print(*bind_site, sep=",", file=res)
else:
    if "1." in proteinECnumber[0:2]:
        print("Your protein is an oxidoreductase, so it should have a cofactor. But it could not be identified. "
              "Please, check uniprot entry " + str(up_maxid) + " -the best hit identified from the M-CSA database- "
                                                               "for more information.")
        print("Your protein is an oxidoreductase, so it should have a cofactor. But it could not be identified. "
              "Please, check uniprot entry " + str(up_maxid) + " -the best hit identified from the M-CSA database- "
                                                               "for more information.", file=res)
    elif "2." in proteinECnumber[0:2]:
        print("Your protein is a transferase, commonly, cofactor dependant. Please, check uniprot entry "
              + str(up_maxid) + " -the best hit identified- for more information.")
        print("Your protein is a transferase, commonly, cofactor dependant. Please, check uniprot entry "
              + str(up_maxid) + " -the best hit identified- for more information.", file=res)
    elif "3." in proteinECnumber[0:2]:
        print("Your protein is an hydrolase.")
        print("Your protein is an hydrolase.", file=res)
    elif "4." in proteinECnumber[0:2]:
        print("Your protein is most similar to characterised lyases.")
        print("Your protein is most similar to characterised lyases.", file=res)
    elif "5." in proteinECnumber[0:2]:
        print("Your protein seems to be an isomerase.")
        print("Your protein seems to be an isomerase.", file=res)
    elif "6." in proteinECnumber[0:2]:
        print("Your protein seems to be an ligases. Normally, this enzymes are ATP dependant but no cofactor could "
              "be identified. Please, check uniprot entry " + str(up_maxid) +
              " -the best hit identified- for more information.")
        print("Your protein seems to be an ligases. Normally, this enzymes are ATP dependant but no cofactor could "
              "be identified. Please, check uniprot entry " + str(up_maxid) +
              " -the best hit identified- for more information.", file=res)

res.close()

time.sleep(3)

# Clean the directory of intermediate files which are not necessary
os.remove("refsequp.dnd")
tokeep = ["query.fasta", str(up_maxid)+".fasta"]
for file in glob.glob("*.fasta"):
    if file not in tokeep:
        os.remove(file)

print("\nFinished running the ActiveSiteID!\n. Your results can be found in " + str(os.getcwd()) +
      ", in a file named Active_site.txt, together with the alignment and the fasta files for your sequences. "
      "You can run now the Surface&Clusters module! ")

if SHOW_STRUCTURE == True:
    import py3Dmol
    residues = []
    with open('Active_site.txt', "r") as f:
        for line in f.readlines():
            if "-->" in line:
                toadd = re.findall(r'\d+', line)
                residues.append(int(toadd[0]))

    view = py3Dmol.view()
    try:
        view.addModel(open('../Modeller/query_multimeric.pdb','r').read(),'pdb')
    except FileNotFoundError:
        view.addModel(open('../Modeller/query_monomeric.pdb','r').read(),'pdb')
    view.setStyle({'model':-1},{'cartoon': {'color':'blue'}})
    view.setStyle({'resi': residues}, {"stick": {'color': 'red'}})
    view.zoomTo()
    view.show()

In [ ]:
#@title MODULE 3 - SURFACE ANALYSIS

#@markdown Determine the distance between any user defined residue in your structure and the different clusters.
import random

#@markdown Indicate the full path to the pdb file you want to analyse. Ex. /content/test/Modeller/query_quaternary.pdb

PDB_FILE = "/content/TEST/Modeller/query_multimeric.pdb" #@param {type:"string"}
#@markdown Indicate if you have run before the ACTIVE SITE module in the same project.
ACTIVE_SITE = True #@param {type:"boolean"}
import numpy as np
from Bio.PDB import HSExposureCB

pdb_directory = os.path.abspath((os.path.join(PDB_FILE, os.pardir)))
working_directory = os.path.abspath((os.path.join(pdb_directory, os.pardir)))
%cd $working_directory
if os.path.exists("Surface") is False:
    os.mkdir("Surface")

os.chdir("./Surface")

try:
    shutil.copy(PDB_FILE, './query.pdb')
except BaseException:
    print('Cannot find the pdb file. Check again the name!')
    quit()

# Open the desired pdb file
parser = PDBParser()
fullid = "Q00F"
fullfile = "query.pdb"
full_structure = parser.get_structure(fullid, fullfile)
mdel = full_structure[0]
ppb = PPBuilder()

ca_coord = []
number_of_chains = 0
for chain in mdel.get_list():
    number_of_chains += 1
for residue in chain.get_list():
    if residue.has_id("CA"):
        ca = residue["CA"]
        ca_coord.append(ca.get_coord())

print("\nMeasuring the minimum bounding box...\n")

ca_coord = np.array(ca_coord)
min_x = min(ca_coord[:, 0])
min_y = min(ca_coord[:, 1])
min_z = min(ca_coord[:, 2])
max_x = max(ca_coord[:, 0])
max_y = max(ca_coord[:, 1])
max_z = max(ca_coord[:, 2])
box_dimensions = ([max_x - min_x, max_y - min_y, max_z - min_z])
time.sleep(2)
volume = box_dimensions[0] * box_dimensions[1] * box_dimensions[2]
print("Your protein, formed by " + str(number_of_chains) + " chains, has an aproximate dimensions of: "
  + str(box_dimensions[0]) + " A, " + str(box_dimensions[1]) + " A, " + str(box_dimensions[2]) +
  " A.\nGiving an aproximate volume of: " + str(volume) + " A\u00b3\n")


print("Calculating the exposure of each residue in the structure...")
RADIUS = 16.0
RADIUS = 16.0
hse_cb = HSExposureCB(mdel, radius=RADIUS)
buried = []
semiburied = []
exposed = []
print("Sorting the residues depending on their exposure...")
for r in mdel.get_residues():
    if is_aa(r) and r.xtra["EXP_HSE_B_U"] >= 35 and r.xtra["EXP_HSE_B_D"] >= 35:
        buried.append(r)
    elif is_aa(r) and 35 > r.xtra["EXP_HSE_B_U"] >= 25 and 35 > r.xtra["EXP_HSE_B_D"] >= 25:
        semiburied.append(r)
    elif is_aa(r):
        exposed.append(r)

# Calculate the surface in A according to the predicted surface of each amino acid classified as exposed
# - 10.1371/journal.pone.0080635
aa = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO",
  "SER", "THR", "TRP", "TYR", "VAL"]
aa_s2 = {}
aa_n = {}
for aaname in aa:
    aa_s2.update({aaname: int()})
    aa_n.update({aaname: int()})

aa_ASA = {"ALA": 121, "ARG": 265, "ASN": 187, "ASP": 187, "CYS": 148, "GLU": 214, "GLN": 214, "GLY": 97,
      "HIS": 216, "ILE": 195, "LEU": 191, "LYS": 230, "MET": 203, "PHE": 228, "PRO": 154, "SER": 143,
      "THR": 163, "TRP": 265, "TYR": 255, "VAL": 165}
totals2 = 0
for r in exposed:
    totals2 += aa_ASA[r.get_resname()]
i = 0
for i in range(20):
    if r.get_resname() == aa[i]:
        aa_s2[aa[i]] += aa_ASA[aa[i]]
        aa_n[aa[i]] += 1
    else:
        i += 1

aa_s2_perc = {}
for aaname in aa:
    aa_s2_perc.update({aaname: str("{:.2f}".format(((aa_s2[aaname] / totals2) * 100)) + "%")})

print("Total number of surface residues: " + str(len(exposed)) + " out of " + str(len(exposed) + len(semiburied) + len(buried)))

if os.path.exists("General_info.txt"):
    check = open("General_info.txt").read()
else:
    check = ""
if "As for" not in check:
    with open("General_info.txt", "a") as f1:
        print("Your protein aproximate dimensions in Angstroms are: " + str(box_dimensions[0]) + "A, " +
              str(box_dimensions[1]) + "A, " + str(
            box_dimensions[2]) + "A.\nWhich makes an aproximate volume of: " +
              str(volume) + " A\u00b3\\n", file=f1)
        print("\nAs for its surface, it measures " + str(totals2) + " A\u00b2\\n", file=f1)
        print("From which each amino acid contributes to:", file=f1)
        print("Aminoacid\t Surface(%)\t Total number", file=f1)
        for aa in aa_s2_perc:
            print(aa + "\t\t" + aa_s2_perc[aa] + "\t\t" + str(aa_n[aa]), file=f1)
        print("Total number of surface residues: " + str(len(exposed)), file=f1)


aatocheck = ["LYS", "GLU", "ASP", "HIS", "CYS", "TYR", "ARG"]  # commonly used for immobilisation#
hydrophobic = ["ILE", "LEU", "PHE", "VAL", "TRP", "TYR"]  # arunachalam 2008#
charged = []
hydroph = []

for r in exposed:
    name = r.get_resname()
    if name in aatocheck:
        chainid = r.get_parent().get_id()
        resseq = r.get_id()[1]
        charged.append(name + "_" + str(resseq) + "_" + chainid)
    elif name in hydrophobic:
        chainid = r.get_parent().get_id()
        resseq = r.get_id()[1]
        hydroph.append(name + "_" + str(resseq) + "_" + chainid)

poslist = []
neglist = []
hislist = []
lyslist = []
cyslist = []

for item in charged:
    if "LYS" in item:
        poslist.append(item)
        lyslist.append(item)
    elif "ASP" in item or "GLU" in item:
        neglist.append(item)
    elif "HIS" in item:
        poslist.append(item)
        hislist.append(item)
    elif "CYS" in item:
        cyslist.append(item)
    elif "ARG" in item:
        poslist.append(item)

def clustering(list1, dic):
    # This method checks if the residues in a certain list have their CA at less than 10A.
    # If so, this residues will be included in the dictionary as a new cluster
    full_structure = parser.get_structure(fullid, fullfile)
    i = 0
    while i in range(len(list1)):
        x1 = list1[i]
        for x2 in list1:
            if x1 != x2:
                dis = mdel[x1.split("_")[2]][int(x1.split("_")[1])]["CA"] - \
                      mdel[x2.split("_")[2]][int(x2.split("_")[1])]["CA"]
                if dis < 10:
                    try:
                        dic[x1].append(x2)
                    except KeyError:
                        dic[x1] = [x2]
        i += 1

def clean(dict1):
    # Clean up the dictionaries created with clustering. This avoids repetition of the same cluster as well as
    # deletes any cluster formed by less than 3 different aminoacids
    dictcopy = dict1.copy()
    keys = list(dictcopy.keys())
    for key in keys:
        if key in dict1[key]:
            dict1.pop(key)
        elif len(dict1[key]) < 2:
            dict1.pop(key)
        elif key in str(dict1.values()):
            dict1.pop(key)
        elif "" in dict1[key]:
            while "" in dict1[key]:
                dict1[key].remove("")

def writeclusterstocsv(cluster, file):
    # Self explanatory. Writes the clusters identified into a csv file.
    with open(file, "a", newline='') as f:
        writer = csv.writer(f)
        if len(cluster) > 1:
            for i in cluster:
                towrite = [i]
                for x in cluster[i]:
                    towrite.append(x)
                writer.writerow(towrite)
            writer.writerow("\n")

def show_clusters(dict):
    # Writes the information of teh clusters into a txt file. More "user friendly" explanation of the results
    f = open("Clusters.txt", "a")
    if len(dict) > 1:
        f.write(list(dict.keys())[0] + " residues in a cluster (<10A):\n")
        i = 1
        for keys in list(dict.keys())[1:]:
            toprint = "Cluster " + str(list(dict.keys())[0][0:3].lower()) + str(i) + ":" + str(keys)
            length = len(dict[keys])
            x = 0
            while x < length:
                toprint += ", " + str(dict[keys][x])
                x += 1
            toprint += "."
            print(toprint, file=f)
            i += 1
    f.write("\n")

def uploadcluster():
    # In case the clusters.csv (which is created in the first execution -  is available, file is opened and clusters
    # written back into a Python dictionary to continue execution
    csv_clusters = csv.reader(open("clusters.csv", "r"))
    i = 0
    keys = [[], [], [], [], [], []]
    values = [[], [], [], [], [], []]
    with open("clusters.csv", "r") as file:
        csv_clusters = csv.reader(file)
        for row in csv_clusters:
            if row[0] == "\n":
                i += 1
            else:
                keys[i].append(row[0])
                values[i].append(row[1:])
    x = 0
    for x in range(len(keys)):
        try:
            if keys[x][0] == "Positive":
                for y in range(len(keys[x])):
                    cluster_pos[keys[0][y]] = values[x][y]
            elif keys[x][0] == "Negative":
                for y in range(len(keys[x])):
                    cluster_neg[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Histidine":
                for y in range(len(keys[x])):
                    cluster_his[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Lysine":
                for y in range(len(keys[x])):
                    cluster_lys[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Cysteine":
                for y in range(len(keys[x])):
                    cluster_cys[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Hydrophobic":
                for y in range(len(keys[x])):
                    cluster_hidroph[keys[x][y]] = values[x][y]
        except IndexError:
            x += 1
        x += 1
def clusters_pymol(dict):
    # Writes the clusters into a pymol command which can be direclty copied into the command line
    if len(dict) > 1:
        f = open("PyMol_clusters.pml", "a")
        colours = coloursdic[list(dict.keys())[0]]
        i = 0
        for keys in dict.keys():
            if keys.isalpha() is not True:
                toprint = ("select " + str(list(dict.keys())[0][0:3].lower()) + str(i) + ", (resi " + str(
                    keys.split("_")[1])) + " & chain " + str(dict[keys][1].split("_")[2])
                length = len(dict[keys])
                x = 0
                while x < length:
                    toprint += ", resi " + str(dict[keys][x].split("_")[1]) + " & chain " + str(
                        dict[keys][x].split("_")[2])
                    x += 1
                toprint += ")"
                print(toprint, file=f)
            i += 1
        print("color " + colours + ", " + str(list(dict.keys())[0][0:3].lower()) + "*\n", file=f)


cluster_pos = {"Positive": ["Lys", "Arg", "His"]}
cluster_neg = {"Negative": ["Asp", "Gly"]}
cluster_his = {"Histidine": ["only", "histidines"]}
cluster_lys = {"Lysine": ["only", "lysines"]}
cluster_cys = {"Cysteine": ["only", "cysteines"]}
cluster_hidroph = {"Hydrophobic": ["ILE", "LEU", "PHE", "VAL", "TRP", "TYR"]}
coloursdic = {"Positive": "Blue", "Negative": "Red", "Histidine": "Cyan", "Lysine": "LightBlue",
              "Cysteine": "Yellow", "Hydrophobic": "White"}

if os.path.exists("clusters.csv") is True:
    print("Uploading previously calculated clusters form clusters.csv.\n")
    uploadcluster()
elif os.path.exists("clusters.csv") is False:
    print("Measuring distances and identifying clusters...")
    clustering(poslist, cluster_pos)
    clustering(neglist, cluster_neg)
    clustering(hislist, cluster_his)
    clustering(lyslist, cluster_lys)
    clustering(cyslist, cluster_cys)
    clustering(hydroph, cluster_hidroph)
    clean(cluster_pos)
    clean(cluster_neg)
    clean(cluster_his)
    clean(cluster_lys)
    clean(cluster_cys)
    clean(cluster_hidroph)
    writeclusterstocsv(cluster_pos, "clusters.csv")
    writeclusterstocsv(cluster_neg, "clusters.csv")
    writeclusterstocsv(cluster_his, "clusters.csv")
    writeclusterstocsv(cluster_lys, "clusters.csv")
    writeclusterstocsv(cluster_cys, "clusters.csv")
    writeclusterstocsv(cluster_hidroph, "clusters.csv")
    show_clusters(cluster_pos)
    show_clusters(cluster_neg)
    show_clusters(cluster_his)
    show_clusters(cluster_lys)
    show_clusters(cluster_cys)
    show_clusters(cluster_hidroph)
    print("Clusters can be found in Clusters.txt")

with open("PyMol_clusters.pml", "w") as f:
    chainid = []
    for chains in full_structure.get_chains():
        if chains.get_id() not in chainid:
            chainid.append(chains.get_id())
    if len(chains) > 1:
        for chain in chainid:
            print("set_color shade" + str(chain) + "= [1.0, " + str(random.random()) + ", " + str(
                random.random()) + "]", file=f)
            print("colour shade" + str(chain) + ", chain " + str(chain), file=f)
clusters_pymol(cluster_pos)
clusters_pymol(cluster_neg)
clusters_pymol(cluster_his)
clusters_pymol(cluster_lys)
clusters_pymol(cluster_cys)
clusters_pymol(cluster_hidroph)

def distointer(cluster, list1):
    # Check if the residues in the cluser can interfere with the protein interface
    itemslist = []
    for kcluster in list(cluster.keys())[1:]:
        itemslist.append(kcluster)
        for values in cluster[kcluster]:
            itemslist.append(values)
    for pos1 in itemslist:
        for pos2 in interface_res["A"]:
            dis = mdel[str(pos1.split("_")[2])][int(pos1.split("_")[1])]["CA"] - mdel["A"][pos2]["CA"]
            if dis != 0 and dis <= 10 and (pos1 not in list1):
                list1.append(pos1)
            if IndexError:
                continue

def writecluster(original, d1, list1, file):
    # Similar to show_cluster method, it writes the information into a .txt file. In this case, it will add a
    # -Warning- if the cluster is close to the specified residues.'''
    it = 1
    f = open(file, "a")
    with open(file) as readfile:
        if "Chain" not in readfile.read():
            f.write("Searching for clusters in close contact to: ")
            if isinstance(original, dict) is True:
                for aas in original:
                    f.write("\nChain " + str(aas) + ":")
                    for aas2 in original[aas]:
                        f.write(" - " + str(aas2))
            elif isinstance(original, list):
                for aas in original:
                    f.write(" - " + str(aas))
            readfile.close()
        else:
            readfile.close()
    f.write("\n" + list(d1.keys())[0] + " residues in a cluster (<10A):\n")
    inlist1 = []
    for k in list(d1.keys())[1:]:
        toprint = "Cluster " + str(list(d1.keys())[0][0:3].lower()) + str(it) + ":" + str(k)
        length = len(d1[k])
        inlist1.append(k)
        x = 0
        while x < length:
            toprint += ", " + str(d1[k][x])
            inlist1.append(d1[k][x])
            x += 1
        toprint += "."
        for ex in inlist1:
            if ex in list1 and "\t -In close proximity to specified residue/s - " not in toprint:
                toprint += "\t -In close proximity to specified residue/s - "
        print(toprint, file=f)
        it += 1
    f.write("\n")
    f.close()

# Start first, identifying how many chains does the model have. Maximum of 10 for convenience
# (and because most proteins fall into this category).
chainres = {"A": [], "B": [], "C": [], "D": [], "E": [], "F": [], "G": [], "H": [], "I": [], "J": []}
chainids = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]

# number of chains in model
number_of_chains = 0
for chain in mdel.get_chains():
    number_of_chains += 1

# Create dictionary with all CA location of a chain
for x in range(number_of_chains):
    for residues in mdel[chainids[x]]:
        if is_aa(residues) is True:
            chainres[chainids[x]].append(residues["CA"])

# Delete unnecessary keys in the chainres dictionary
for a in range(len(chainres.keys())):
    if len(chainres[chainids[a]]) == 0:
        del chainres[chainids[a]]

# calculation of the interface residues. This will assume all chains will have the same interactions as chain A.
# This simplification is necessary to get results in a reasonable time.
interface_res = {"A": []}

if number_of_chains > 1:
    print("Calculating if any of the clusters might affect the quaternary structure of your protein. \n "
          "Note: This assumes chain A is in contact with all other subunits and all contacts are identical "
          "between subunits.\n")

calculationdone = os.path.exists("residues_interface.txt")

print("Calculating contacts between chain A and it's neighbours...")
ch_1 = 1
# Calculate the distance of all atoms in chain A to all atoms in other chains. If this distance is <10A,
# the position will be added as part of the interface.
while ch_1 < number_of_chains:
    for ca1 in chainres["A"]:
        for ca2 in chainres[chainids[ch_1]]:
            if ca1.get_parent().get_id()[1] not in interface_res["A"]:
                dist = ca1 - ca2
                if 0 < dist < 10:
                    interface_res["A"].append(ca1.get_parent().get_id()[1])
    ch_1 += 1

    # Assuming all contacts in the different chains are the same as in chain A,
    # copy the residue position to each chain.
    count_start = 1
    for count_start in range(number_of_chains):
        interface_res[chainids[count_start]] = interface_res["A"]
        count_start += 1

    # Write information to file so it doesn't have to be calculated again.
    with open("residues_interface.txt", "a") as fileint:
        print(interface_res, file=fileint)
    print("The position of the residues identified as part of the interface can be found in residues_interface.txt")

# Calculate now if any of the identified clusters is at less than 10A to the any of the interface residues.
intpos = []
intneg = []
inthis = []
intlys = []
intcys = []
inthidroph = []

distointer(cluster_pos, intpos)
writecluster(interface_res, cluster_pos, intpos, "Interface_contacts.txt")
distointer(cluster_neg, intneg)
writecluster(interface_res, cluster_neg, intneg, "Interface_contacts.txt")
distointer(cluster_his, inthis)
writecluster(interface_res, cluster_his, inthis, "Interface_contacts.txt")
distointer(cluster_lys, intlys)
writecluster(interface_res, cluster_lys, intlys, "Interface_contacts.txt")
distointer(cluster_cys, intcys)
writecluster(interface_res, cluster_cys, intcys, "Interface_contacts.txt")
distointer(cluster_hidroph, inthidroph)
writecluster(interface_res, cluster_hidroph, inthidroph, "Interface_contacts.txt")
print("Distance to the multimeric interface successful. Your results can be found in Cluster_interface.txt")

# __________________________ACTIVE SITE + CLUSTERS _________________________________________________________________
#   
def dist_to_as(dict1, list1):
    # Similarly to the other distance methods, calculates if any residues of the cluster is at less than
    # 10A of any of the residues defined as the active site'''
    cluster_aa = []
    for k in list(dict1.keys())[1:]:
        cluster_aa.append(k)
        for i in dict1[k]:
            cluster_aa.append(i)
    for as_site in aspositions:
        for x in cluster_aa:
            for i in range(number_of_chains):
                dis = mdel[chainids[i]][int(as_site)]["CA"] - mdel[x.split("_")[2]][int(x.split("_")[1])]["CA"]
                if dis < 5:
                    if x not in list1:
                        list1.append(x)

if ACTIVE_SITE == True:
    # If this information exists in the active_site directory, give the option to use directly that information.
    # If not, ask for user input on where the active site is located
    parent_dir = os.path.dirname(os.getcwd())
    str_ac = parent_dir + "/ActiveSite/Active_site.txt"
    aspositions_0 = []
    if os.path.exists(str_ac) is True:
        with open(parent_dir + "/ActiveSite/Active_site.txt", "r") as f:
            for line in f.readlines():
                if "-->" in line:
                    toadd = re.findall(r'\d+', line)
                    aspositions_0.append(toadd)
    else:
        print("Cannot find the active site identified with the ActiveSiteID module, "
              "please run it and check the results!")
    
aspositions = []
for aa in aspositions_0:
    if isinstance(aa, list) is True:
        for aa1 in aa:
            aspositions.append(aa1)
    else:
        aspositions.append(aa)

    # Calculate if any residue in the cluster is close to the active site residues
asposd = []
asnegd = []
ashisd = []
aslysd = []
ascysd = []
ashidrophd = []

number_of_chains = 0
for chain in mdel.get_chains():
    number_of_chains += 1
    
dist_to_as(cluster_pos, asposd)
dist_to_as(cluster_neg, asnegd)
dist_to_as(cluster_his, ashisd)
dist_to_as(cluster_lys, aslysd)
dist_to_as(cluster_cys, ascysd)
dist_to_as(cluster_hidroph, ashidrophd)
writecluster(aspositions, cluster_pos, asposd, "Cluster_ActiveSite.txt")
writecluster(aspositions, cluster_neg, asnegd, "Cluster_ActiveSite.txt")
writecluster(aspositions, cluster_his, ashisd, "Cluster_ActiveSite.txt")
writecluster(aspositions, cluster_lys, aslysd, "Cluster_ActiveSite.txt")
writecluster(aspositions, cluster_cys, ascysd, "Cluster_ActiveSite.txt")
writecluster(aspositions, cluster_hidroph, ashidrophd, "Cluster_ActiveSite.txt")
print("Information of the cluster distance to the active site can be found in Cluster_ActiveSite.txt")

In [ ]:
#@markdown Show 3D structure with the clusters
SURFACE_FOLDER = "/content/TEST/Surface" #@param {type:"string"}
#@markdown Indicate the full path to your Surface folder. Ex. /content/test/Surface
FILE_CLUSTERS = SURFACE_FOLDER + '/clusters.csv'

import py3Dmol

def uploadcluster(FILE):
    # In case the clusters.csv (which is created in the first execution -  is available, file is opened and clusters
    # written back into a Python dictionary to continue execution
    csv_clusters = csv.reader(open(FILE, "r"))
    i = 0
    cluster_pos, cluster_neg, cluster_his, cluster_lys, cluster_cys, cluster_hidroph = {}, {} ,{} ,{} ,{} ,{}
    keys = [[], [], [], [], [], []]
    values = [[], [], [], [], [], []]
    with open(FILE, "r") as file:
        csv_clusters = csv.reader(file)
        for row in csv_clusters:
            if row[0] == "\n":
                i += 1
            else:
                keys[i].append(row[0])
                values[i].append(row[1:])

    for x in range(len(keys)):
        try:
            if keys[x][0] == "Positive":
                for y in range(len(keys[x])):
                    cluster_pos[keys[0][y]] = values[x][y]
            elif keys[x][0] == "Negative":
                for y in range(len(keys[x])):
                    cluster_neg[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Histidine":
                for y in range(len(keys[x])):
                    cluster_his[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Lysine":
                for y in range(len(keys[x])):
                    cluster_lys[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Cysteine":
                for y in range(len(keys[x])):
                    cluster_cys[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Hydrophobic":
                for y in range(len(keys[x])):
                    cluster_hidroph[keys[x][y]] = values[x][y]
        except IndexError:
            x += 1
        x += 1
    return cluster_pos, cluster_neg, cluster_his, cluster_lys, cluster_cys, cluster_hidroph


def num_cluster(DICT):
  list1 = []
  for k in list(DICT.keys()):
    if '_' in k:
      list1.append(k.split('_')[1])
  list2 = []
  for v_list in list(DICT.values()):
      for v in v_list:
          if '_' in v:
              list2.append(v.split('_')[1])
  list3 = list1 + list2
  return list3



if os.path.exists(FILE_CLUSTERS) is True:
    r_cluster_pos, r_cluster_neg, r_cluster_his, r_cluster_lys, r_cluster_cys, r_cluster_hidrop = uploadcluster(FILE_CLUSTERS)
    cluster_pos = num_cluster(r_cluster_pos)
    cluster_neg = num_cluster(r_cluster_neg)
    cluster_his = num_cluster(r_cluster_his)
    cluster_lys = num_cluster(r_cluster_lys)
    cluster_cys = num_cluster(r_cluster_cys)
    cluster_hidrop = num_cluster(r_cluster_hidrop)
elif os.path.exists(FILE_CLUSTERS) is False:
    print('Cannot find the clutsers.csv file.. make sure you pointed to the correct path!')

import py3Dmol
view = py3Dmol.view()
try:
    view.addModel(open(SURFACE_FOLDER + '/query.pdb','r').read(),'pdb')
except FileNotFoundError:
    print('There is no pdb file in the Surface folder... and it should be named query.pdb!')

view.setStyle({'model':-1},{'cartoon': {'color':'white'}})
view.setBackgroundColor('black')
view.setStyle({'resi': cluster_pos}, {"sphere": {'color': 'blue'}})
view.setStyle({'resi': cluster_neg}, {"sphere": {'color': 'red'}})
view.setStyle({'resi': cluster_his}, {"sphere": {'color': 'cyan'}})
view.setStyle({'resi': cluster_lys}, {"sphere": {'color': 'lightblue'}})
view.setStyle({'resi': cluster_cys}, {"sphere": {'color': 'yellow'}})
view.setStyle({'resi': cluster_hidrop}, {"sphere": {'color': 'white'}})

view.zoomTo()
view.show()

In [ ]:
#@title MODULE 3.1- CLUSTER DISTANCE
#@markdown Determine the distance between any user defined residue in your structure and the different clusters.

#@markdown Indicate the full path to the Surface folder where the information is.
SURFACE_FOLDER = "/content/TEST/Surface" #@param {type:"string"}
#@markdown Enter the residues to calculate distance to with the number and the chain and separated by a comma. Ex. 1_A, 200_B
RESIDUES =  '50_A' #@param {type:"string"}
#@markdown Enter the file name to save your results. 
FILE_NAME =  'test1' #@param {type:"string"}

%cd $SURFACE_FOLDER
import py3Dmol

residues = RESIDUES.split(',')
pdbfile = SURFACE_FOLDER + '/query.pdb'

# Open the desired pdb file
parser = PDBParser()
fullid = "Q00F"
pdbfiles = []
fullfile = pdbfile
full_structure = parser.get_structure(fullid, fullfile)
mdel = full_structure[0]
ppb = PPBuilder()

def uploadcluster(FILE):
    # In case the clusters.csv (which is created in the first execution -  is available, file is opened and clusters
    # written back into a Python dictionary to continue execution
    i = 0
    cluster_pos, cluster_neg, cluster_his, cluster_lys, cluster_cys, cluster_hidroph = {}, {} ,{} ,{} ,{} ,{}
    keys = [[], [], [], [], [], []]
    values = [[], [], [], [], [], []]
    with open(FILE, "r") as file:
        csv_clusters = csv.reader(file)
        for row in csv_clusters:
            if row[0] == "\n":
                i += 1
            else:
                keys[i].append(row[0])
                values[i].append(row[1:])
    x = 0
    for x in range(len(keys)):
        try:
            if keys[x][0] == "Positive":
                for y in range(len(keys[x])):
                    cluster_pos[keys[0][y]] = values[x][y]
            elif keys[x][0] == "Negative":
                for y in range(len(keys[x])):
                    cluster_neg[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Histidine":
                for y in range(len(keys[x])):
                    cluster_his[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Lysine":
                for y in range(len(keys[x])):
                    cluster_lys[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Cysteine":
                for y in range(len(keys[x])):
                    cluster_cys[keys[x][y]] = values[x][y]
            elif keys[x][0] == "Hydrophobic":
                for y in range(len(keys[x])):
                    cluster_hidroph[keys[x][y]] = values[x][y]
        except IndexError:
            x += 1
        x += 1
    return cluster_pos, cluster_neg, cluster_his,cluster_lys, cluster_cys, cluster_hidroph
def num_cluster(DICT):
    list1 = []
    for k in list(DICT.keys()):
      if '_' in k:
        list1.append(k.split('_')[1])
    list2 = []
    for v_list in list(DICT.values()):
        for v in v_list:
            if '_' in v:
                list2.append(v.split('_')[1])
    list3 = list1 + list2
    return list3


def distonrl(cluster):
    # Similarly to the other distance methods, calculates if any residues of the cluster is at less than 10A
    # of any of the residues defined by the user
    itemslist = []
    list1=[]
    for k0 in list(cluster.keys())[1:]:
        itemslist.append(k0)
        for values in cluster[k0]:
            itemslist.append(values)
    for i1 in itemslist:
        for k1 in nrl_dic.keys():
            for v0 in nrl_dic[k1]:
                dis = mdel[str(i1.split("_")[2])][int(i1.split("_")[1])]["CA"] - mdel[str(k1)][int(v0)]["CA"]
                if dis != 0 and dis < 10 and i1 not in list1:
                    list1.append(i1)
                elif IndexError:
                    continue
    return list1

def writecluster(original, d1, list1, file):
    # Writes the information into a .txt file. In this case, it will add a -Warning- if the cluster is close
    # to the specified residues.
    if len(d1) > 0:
        it = 1
        f = open(file, "a")
        with open(file) as readfile:
            if "Chain" not in readfile.read():
                f.write("Searching for clusters in close contact to: ")
                if isinstance(original, dict) is True:
                    for aas in original:
                        f.write("\nChain " + str(aas) + ":")
                        for aas2 in original[aas]:
                            f.write(" - " + str(aas2))
                elif isinstance(original, list):
                    for aas in original:
                        f.write(" - " + str(aas))
                readfile.close()
            else:
                readfile.close()
        f.write("\n" + list(d1.keys())[0] + " residues in a cluster (<10A):\n")
        inlist1 = []
        for k in list(d1.keys())[1:]:
            toprint = "Cluster " + str(list(d1.keys())[0][0:3].lower()) + str(it) + ":" + str(k)
            length = len(d1[k])
            inlist1.append(k)
            x = 0
            while x < length:
                toprint += ", " + str(d1[k][x])
                inlist1.append(d1[k][x])
                x += 1
            toprint += "."
            for ex in inlist1:
                if ex in list1 and "\t -In close proximity to specified residue/s - " not in toprint:
                    toprint += "\t -In close proximity to specified residue/s - "
            print(toprint, file=f)
            it += 1
        f.write("\n")
        f.close() 


def distance_pml(dict, file):
    filename= FILE_NAME + '.pml'
    # Writes the clusters into a pymol command which can be directly copied into the command line
    for key in dict.keys():
        writing = 'select res_group ('
        j=0
        item_n = 0
        for aa_n in dict[key]:
            length = len(dict[key])
            if item_n == 0:
                writing += 'resi ' + aa_n + ' & chain ' + key
                item_n += 1
            else:
                writing += ', resi ' + aa_n + ' & chain ' + key
                item_n += 1
        writing += ')\n'
        writing += 'zoom (res_group)'
        with open(filename, "w") as f:
            f.write(writing)
    return filename


# Extract from the pdb file the chain id and the sequence of each chain saved into a dictionary.

cluster_pos, cluster_neg, cluster_his,cluster_lys, cluster_cys, cluster_hidrop = uploadcluster(SURFACE_FOLDER + '/clusters.csv')
num_cluster_pos = num_cluster(cluster_pos)
num_cluster_neg = num_cluster(cluster_neg)
num_cluster_his = num_cluster(cluster_his)
num_cluster_lys = num_cluster(cluster_lys)
num_cluster_cys = num_cluster(cluster_cys)
num_cluster_hidrop = num_cluster(cluster_hidroph)

c_ids = {}
for chains in mdel:
    c_ids[chains.get_id()] = []
for chains in c_ids:
    c_ids[chains] = [ppb.build_peptides(mdel[chains])[0][0].get_id()[1],
                     ppb.build_peptides(mdel[chains])[0][-1].get_id()[1]]

# Ask user for the specific residues to check. Check that the specified residues exist in the pdb file.
# nrl = new residues list, but I wanted to keep it short.
nrl = residues
for res in nrl:
    if res.split("_")[1] not in c_ids.keys():
        print("\nIt seems that your input is part of an unexistent chain.")
        quit()
    elif int(res.split("_")[0]) not in range(c_ids[res.split("_")[1]][0], c_ids[res.split("_")[1]][1]):
        print("\nIt seems that your input is not part of the pdb file.")
        quit()

nrl_dic = {}
for posit in nrl:
    if posit.split("_")[1] in nrl_dic.keys():
        nrl_dic[posit.split("_")[1]].append(posit.split("_")[0])
    else:
        nrl_dic[posit.split("_")[1]] = [posit.split("_")[0]]

perspos = distonrl(cluster_pos)
persneg = distonrl(cluster_neg)
pershis = distonrl(cluster_his)
perslys = distonrl(cluster_lys)
perscys = distonrl(cluster_cys)
pershidroph = distonrl(cluster_hidroph)

spfile = FILE_NAME

writecluster(nrl_dic, cluster_pos, perspos, spfile + ".txt")
writecluster(nrl_dic, cluster_neg, persneg, spfile + ".txt")
writecluster(nrl_dic, cluster_his, pershis, spfile + ".txt")
writecluster(nrl_dic, cluster_lys, perslys, spfile + ".txt")
writecluster(nrl_dic, cluster_cys, perscys, spfile + ".txt")
writecluster(nrl_dic, cluster_hidrop, pershidroph, spfile + ".txt")

num_dist_pos = [x.split('_')[1] for x in perspos]
num_dist_neg = [x.split('_')[1] for x in persneg]
num_dist_his = [x.split('_')[1] for x in pershis]
num_dist_lys = [x.split('_')[1] for x in perslys]
num_dist_cys = [x.split('_')[1] for x in perscys]
num_dist_hidrop = [x.split('_')[1] for x in pershidroph]

contact_list = num_dist_pos + num_dist_his + num_dist_lys + num_dist_cys + num_dist_hidrop

pml_file = distance_pml(nrl_dic, FILE_NAME)

nrl_list = [x.split('_')[0] for x in nrl]
import py3Dmol
view = py3Dmol.view()
try:
    view.addModel(open(SURFACE_FOLDER + '/query.pdb','r').read(),'pdb')
except FileNotFoundError:
    print('There is no pdb file in the Surface folder... and it should be named query.pdb!')

view.setStyle({'model':-1},{'cartoon': {'color':'white'}})
view.setBackgroundColor('black')
view.setStyle({'resi': num_cluster_pos}, {"sphere": {'color': 'blue'}})
view.setStyle({'resi': num_cluster_neg}, {"sphere": {'color': 'red'}})
view.setStyle({'resi': num_cluster_his}, {"sphere": {'color': 'cyan'}})
view.setStyle({'resi': num_cluster_lys}, {"sphere": {'color': 'lightblue'}})
view.setStyle({'resi': num_cluster_cys}, {"sphere": {'color': 'yellow'}})
view.setStyle({'resi': num_dist_hidrop}, {"sphere": {'color': 'white'}})
view.setStyle({'resi': contact_list}, {"sphere": {'color': 'magenta'}})
view.setStyle({'resi': nrl_list}, {"sphere": {'color': 'purple'}})


view.zoomTo()
view.show()


In [ ]:
#@title Download your results:
#@markdown Just indicate your project folder with the full path (Ex. /content/TEST) to donwload all results in a zip file. 
PROJECT = "TEST" #@param {type:"string"}

os.chdir('/content/')
PROJECT_NAME = PROJECT.split('/')[-1] + '.gz.tar'

!tar -zcf $PROJECT_NAME $PROJECT

from google.colab import files
files.download(PROJECT_NAME) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>